In [2]:
!pip install tensorflow mlflow scikit-learn pyarrow numpy tensorflow keras scikeras numpy pandas==2.2.0 model2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.2/700.2 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold  # Adicionado StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score
from sklearn.preprocessing import LabelEncoder  # Importado LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizers import l2
import time
from sklearn.metrics import make_scorer #importa o make_scorer
from sklearn.model_selection import cross_val_score

# 1. Carregar e preparar os dados
print("Carregando dados...")
df_train = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/dataset_train_trim_synthetic_balanced.parquet')

if 'target' not in df_train.columns:
    df_train['target'] = df_train['sentiment']

# Codificar labels
sentiment_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
y = df_train['target'].map(sentiment_mapping)
X = df_train['comment_cleaned']

# Split dos dados
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# 2. Vetorização do texto
print("Vetorizando textos...")
vectorizer = TfidfVectorizer(
    max_features=8000,
    ngram_range=(1, 2),
    stop_words='english'
)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Converter para arrays densos
X_train_vec = X_train_vec.toarray()
X_test_vec = X_test_vec.toarray()

# 3. Função para criar modelo
def create_model(units1=64, units2=32, dropout_rate=0.3,
                learning_rate=0.001, optimizer='adam',
                activation='relu', regularizer=None):
    model = Sequential()
    # Exemplo de matriz de custos (penalizando mais falsos positivos para a classe 0 - Negativo)
    class_weights = {0: 5, 1: 1, 2: 1} # Peso 5 para a classe Negativo, 1 para as outras
    # Camada de entrada
    model.add(Dense(
        units=units1,
        input_dim=X_train_vec.shape[1],
        activation=activation,
        kernel_regularizer=regularizer
    ))
    model.add(Dropout(dropout_rate))

    # Camada oculta
    model.add(Dense(
        units=units2,
        activation=activation,
        kernel_regularizer=regularizer
    ))
    model.add(Dropout(dropout_rate))

    # Camada de saída
    model.add(Dense(3, activation='softmax'))

    # Compilar modelo
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    else:
        opt = RMSprop(learning_rate=learning_rate)

    model.compile(
        loss='sparse_categorical_crossentropy',  # Importante: para labels inteiros
        optimizer=opt,
        metrics=['accuracy']
    )

    return model


# 4. Configuração da Grid Search
print("\nConfigurando Grid Search...")
param_grid = {
    'units1': [128],
    'units2': [32],
    'dropout_rate': [0.2, 0.3],
    'learning_rate': [0.001, 0.0001],
    'optimizer': ['adam'],
    'batch_size': [32, 64],
    'epochs': [20],
    'activation': ['relu'],
    'regularizer': [None, l2(0.01)]
}

# 5. Grid Search Manual
print("\nIniciando Grid Search...")
start_time = time.time()

best_score = -1
best_params = {}
best_model = None

# Para cada combinação de parâmetros
for units1 in param_grid['units1']:
    for units2 in param_grid['units2']:
        for dropout_rate in param_grid['dropout_rate']:
            for learning_rate in param_grid['learning_rate']:
                for optimizer_name in param_grid['optimizer']:
                    for batch_size in param_grid['batch_size']:
                        for epochs in param_grid['epochs']:
                            for activation in param_grid['activation']:
                                for regularizer_type in param_grid['regularizer']:
                                    print(f"\nTestando: units1={units1}, units2={units2}, dropout={dropout_rate}, lr={learning_rate}, opt={optimizer_name}, batch={batch_size}, epochs={epochs}, activation={activation}, regularizer={regularizer_type}")

                                    # Criar o modelo com os parâmetros atuais
                                    model = create_model(
                                        units1=units1,
                                        units2=units2,
                                        dropout_rate=dropout_rate,
                                        learning_rate=learning_rate,
                                        optimizer=optimizer_name,
                                        activation=activation,
                                        regularizer=regularizer_type
                                    )

                                    # Validação cruzada (cross-validation) - StratifiedKFold para manter a proporção das classes
                                    kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)  # Mantém a proporção das classes
                                    scores = []
                                    for train_index, val_index in kfold.split(X_train_vec, y_train):
                                        X_train_fold, X_val_fold = X_train_vec[train_index], X_train_vec[val_index]
                                        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usando .iloc para indexar corretamente
                                        # Treinar o modelo no conjunto de treinamento da dobra (fold)
                                        model.fit(X_train_fold, y_train_fold, batch_size=batch_size, epochs=epochs, verbose=0) # verbose=0 silencia a saída de treinamento

                                        # Fazer a predição no conjunto de validação da dobra
                                        y_pred_fold = np.argmax(model.predict(X_val_fold), axis=-1)  # Convertendo as predições de probabilidade para rótulos de classe
                                        # Calcular o F1-score ponderado para a dobra
                                        # f1 = f1_score(y_val_fold, y_pred_fold, average='weighted')
                                        # scores.append(f1)

                                        recall_negativo = recall_score(y_val_fold, y_pred_fold, average=None, labels=[0])[0]  # labels=[0] para o recall da classe 0 (Negativo)
                                        print(f"Recall (Negativo): {recall_negativo:.4f}")
                                        scores.append(recall_negativo)

                                    # Calcular a média dos scores de validação cruzada
                                    # mean_f1 = np.mean(scores)
                                    # print(f"F1-score médio (validação cruzada): {mean_f1:.4f}")

                                    mean_recall_negativo = np.mean(scores)
                                    print(f"Recall (Negativo) médio (validação cruzada): {mean_recall_negativo:.4f}")


                                    # Se o score for melhor, salvar os parâmetros e o modelo
                                    if mean_recall_negativo > best_score:
                                        best_score = mean_recall_negativo
                                        best_params = {
                                            'units1': units1,
                                            'units2': units2,
                                            'dropout_rate': dropout_rate,
                                            'learning_rate': learning_rate,
                                            'optimizer': optimizer_name,
                                            'batch_size': batch_size,
                                            'epochs': epochs,
                                            'activation': activation,
                                            'regularizer': regularizer_type
                                        }
                                        best_model = model # Salvando o modelo treinado com os melhores parâmetros
                                    print("-" * 30)  # Separador visual


print(f"\nTempo total de execução: {(time.time() - start_time) / 60:.2f} minutos")

# 7. Resultados
print("\nMelhores parâmetros encontrados:")
for param, value in best_params.items():
    print(f"{param}: {value}")

print(f"\nMelhor score de validação: {best_score:.4f}")

# 8. Avaliação no conjunto de teste (usando o melhor modelo treinado)
print("\nAvaliando no conjunto de teste...")

if best_model is not None:  # Certifique-se de que um modelo foi treinado
    y_pred = np.argmax(best_model.predict(X_test_vec), axis=-1) # Convertendo as predições de probabilidade para rótulos de classe
    test_accuracy = accuracy_score(y_test, y_pred)
    test_f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"\nAcurácia no teste: {test_accuracy:.4f}")
    print(f"F1-Score no teste: {test_f1:.4f}")

    # 9. Relatório detalhado
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred, target_names=sentiment_mapping.keys()))

    # 10. Análise por classe (especialmente para negativas)
    print("\nMétricas para classe Negative:")
    neg_mask = (y_test == 0)
    if np.any(neg_mask):  # Garante que haja exemplos da classe negativa
        neg_accuracy = accuracy_score(y_test[neg_mask], y_pred[neg_mask])
        neg_f1 = f1_score(y_test[neg_mask], y_pred[neg_mask], average='weighted')

        print(f"Acurácia para Negative: {neg_accuracy:.4f}")
        print(f"F1-Score para Negative: {neg_f1:.4f}")
    else:
        print("Nenhum exemplo da classe Negative encontrado no conjunto de teste.")

    # 11. Exemplos de erros (falsos negativos)
    false_negatives = X_test[ (y_test == 0) & (y_pred != 0) ]  # Corrigido para usar os índices originais
    if len(false_negatives) > 0:
        print("\nExemplos de falsos negativos:")
        #Recupera o texto original usando os indices originais.
        false_negatives_indices = np.where((y_test == 0) & (y_pred != 0))[0] # Obtem os indices dos falsos negativos.
        for i, index in enumerate(false_negatives_indices[:5]): # Usa os indices para acessar X_test
            print(f"{i+1}. {X.iloc[index]}") # Usa X.iloc para pegar os textos originais.
    else:
        print("Nenhum falso negativo encontrado.")
else:
    print("Nenhum modelo foi treinado (nenhuma combinação de parâmetros foi melhor).")

Melhores parâmetros encontrados:
units1: 128
units2: 32
dropout_rate: 0.3
learning_rate: 0.001
optimizer: adam
batch_size: 32
epochs: 20
activation: relu
regularizer: None

Melhor score de validação: 0.7561

In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 3.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold  # Adicionado StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score
from sklearn.preprocessing import LabelEncoder  # Importado LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizers import l2
import time
from sklearn.metrics import make_scorer #importa o make_scorer
from sklearn.model_selection import cross_val_score

import spacy

# 1. Carregar e preparar os dados
print("Carregando dados...")
df_train = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/dataset_train_trim_synthetic_balanced.parquet')

if 'target' not in df_train.columns:
    df_train['target'] = df_train['sentiment']

# Codificar labels
sentiment_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
y = df_train['target'].map(sentiment_mapping)
X = df_train['comment_cleaned']

# Split dos dados
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# 2. Vetorização do texto
print("Vetorizando textos...")
# vectorizer = TfidfVectorizer(
#     max_features=8000,
#     ngram_range=(1, 2),
#     stop_words='english'
# )
# X_train_vec = vectorizer.fit_transform(X_train)
# X_test_vec = vectorizer.transform(X_test)

# # Converter para arrays densos
# X_train_vec = X_train_vec.toarray()
# X_test_vec = X_test_vec.toarray()


# Carregar um modelo do SpaCy (escolha um que inclua embeddings)
nlp = spacy.load("en_core_web_lg")  # ou "en_core_web_lg" para embeddings maiores

def get_mean_embedding(text):
    doc = nlp(text)
    return doc.vector

X_train_vec = np.array([get_mean_embedding(text) for text in X_train])
X_test_vec = np.array([get_mean_embedding(text) for text in X_test])

# 3. Função para criar modelo
def create_model(units1=64, units2=32, dropout_rate=0.3,
                learning_rate=0.001, optimizer='adam',
                activation='relu', regularizer=None):
    model = Sequential()
    # Exemplo de matriz de custos (penalizando mais falsos positivos para a classe 0 - Negativo)
    class_weights = {0: 5, 1: 1, 2: 1} # Peso 5 para a classe Negativo, 1 para as outras
    # Camada de entrada
    model.add(Dense(
        units=units1,
        input_dim=X_train_vec.shape[1],
        activation=activation,
        kernel_regularizer=regularizer
    ))
    model.add(Dropout(dropout_rate))

    # Camada oculta
    model.add(Dense(
        units=units2,
        activation=activation,
        kernel_regularizer=regularizer
    ))
    model.add(Dropout(dropout_rate))

    # Camada de saída
    model.add(Dense(3, activation='softmax'))

    # Compilar modelo
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    else:
        opt = RMSprop(learning_rate=learning_rate)

    model.compile(
        loss='sparse_categorical_crossentropy',  # Importante: para labels inteiros
        optimizer=opt,
        metrics=['accuracy']
    )

    return model

#Testando: units1=128, units2=32, dropout=0.2, lr=0.001, opt=adam, batch=32, epochs=20, activation=relu, regularizer=None

# 4. Configuração da Grid Search
print("\nConfigurando Grid Search...")
param_grid = {
    'units1': [128],
    'units2': [32],
    'dropout_rate': [0.2],
    'learning_rate': [0.001],
    'optimizer': ['adam'],
    'batch_size': [32],
    'epochs': [20],
    'activation': ['relu'],
    'regularizer': [None]
}

# 5. Grid Search Manual
print("\nIniciando Grid Search...")
start_time = time.time()

best_score = -1
best_params = {}
best_model = None

# Para cada combinação de parâmetros
for units1 in param_grid['units1']:
    for units2 in param_grid['units2']:
        for dropout_rate in param_grid['dropout_rate']:
            for learning_rate in param_grid['learning_rate']:
                for optimizer_name in param_grid['optimizer']:
                    for batch_size in param_grid['batch_size']:
                        for epochs in param_grid['epochs']:
                            for activation in param_grid['activation']:
                                for regularizer_type in param_grid['regularizer']:
                                    print(f"\nTestando: units1={units1}, units2={units2}, dropout={dropout_rate}, lr={learning_rate}, opt={optimizer_name}, batch={batch_size}, epochs={epochs}, activation={activation}, regularizer={regularizer_type}")

                                    # Criar o modelo com os parâmetros atuais
                                    model = create_model(
                                        units1=units1,
                                        units2=units2,
                                        dropout_rate=dropout_rate,
                                        learning_rate=learning_rate,
                                        optimizer=optimizer_name,
                                        activation=activation,
                                        regularizer=regularizer_type
                                    )

                                    # Validação cruzada (cross-validation) - StratifiedKFold para manter a proporção das classes
                                    kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)  # Mantém a proporção das classes
                                    scores = []
                                    for train_index, val_index in kfold.split(X_train_vec, y_train):
                                        X_train_fold, X_val_fold = X_train_vec[train_index], X_train_vec[val_index]
                                        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usando .iloc para indexar corretamente
                                        # Treinar o modelo no conjunto de treinamento da dobra (fold)
                                        model.fit(X_train_fold, y_train_fold, batch_size=batch_size, epochs=epochs, verbose=0) # verbose=0 silencia a saída de treinamento

                                        # Fazer a predição no conjunto de validação da dobra
                                        y_pred_fold = np.argmax(model.predict(X_val_fold), axis=-1)  # Convertendo as predições de probabilidade para rótulos de classe
                                        # Calcular o F1-score ponderado para a dobra
                                        # f1 = f1_score(y_val_fold, y_pred_fold, average='weighted')
                                        # scores.append(f1)

                                        recall_negativo = recall_score(y_val_fold, y_pred_fold, average=None, labels=[0])[0]  # labels=[0] para o recall da classe 0 (Negativo)
                                        print(f"Recall (Negativo): {recall_negativo:.4f}")
                                        scores.append(recall_negativo)

                                    # Calcular a média dos scores de validação cruzada
                                    # mean_f1 = np.mean(scores)
                                    # print(f"F1-score médio (validação cruzada): {mean_f1:.4f}")

                                    mean_recall_negativo = np.mean(scores)
                                    print(f"Recall (Negativo) médio (validação cruzada): {mean_recall_negativo:.4f}")


                                    # Se o score for melhor, salvar os parâmetros e o modelo
                                    if mean_recall_negativo > best_score:
                                        best_score = mean_recall_negativo
                                        best_params = {
                                            'units1': units1,
                                            'units2': units2,
                                            'dropout_rate': dropout_rate,
                                            'learning_rate': learning_rate,
                                            'optimizer': optimizer_name,
                                            'batch_size': batch_size,
                                            'epochs': epochs,
                                            'activation': activation,
                                            'regularizer': regularizer_type
                                        }
                                        best_model = model # Salvando o modelo treinado com os melhores parâmetros
                                    print("-" * 30)  # Separador visual


print(f"\nTempo total de execução: {(time.time() - start_time) / 60:.2f} minutos")

# 7. Resultados
print("\nMelhores parâmetros encontrados:")
for param, value in best_params.items():
    print(f"{param}: {value}")

print(f"\nMelhor score de validação: {best_score:.4f}")

# 8. Avaliação no conjunto de teste (usando o melhor modelo treinado)
print("\nAvaliando no conjunto de teste...")

if best_model is not None:  # Certifique-se de que um modelo foi treinado
    y_pred = np.argmax(best_model.predict(X_test_vec), axis=-1) # Convertendo as predições de probabilidade para rótulos de classe
    test_accuracy = accuracy_score(y_test, y_pred)
    test_f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"\nAcurácia no teste: {test_accuracy:.4f}")
    print(f"F1-Score no teste: {test_f1:.4f}")

    # 9. Relatório detalhado
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred, target_names=sentiment_mapping.keys()))

    # 10. Análise por classe (especialmente para negativas)
    print("\nMétricas para classe Negative:")
    neg_mask = (y_test == 0)
    if np.any(neg_mask):  # Garante que haja exemplos da classe negativa
        neg_accuracy = accuracy_score(y_test[neg_mask], y_pred[neg_mask])
        neg_f1 = f1_score(y_test[neg_mask], y_pred[neg_mask], average='weighted')

        print(f"Acurácia para Negative: {neg_accuracy:.4f}")
        print(f"F1-Score para Negative: {neg_f1:.4f}")
    else:
        print("Nenhum exemplo da classe Negative encontrado no conjunto de teste.")

    # 11. Exemplos de erros (falsos negativos)
    false_negatives = X_test[ (y_test == 0) & (y_pred != 0) ]  # Corrigido para usar os índices originais
    if len(false_negatives) > 0:
        print("\nExemplos de falsos negativos:")
        #Recupera o texto original usando os indices originais.
        false_negatives_indices = np.where((y_test == 0) & (y_pred != 0))[0] # Obtem os indices dos falsos negativos.
        for i, index in enumerate(false_negatives_indices[:5]): # Usa os indices para acessar X_test
            print(f"{i+1}. {X.iloc[index]}") # Usa X.iloc para pegar os textos originais.
    else:
        print("Nenhum falso negativo encontrado.")
else:
    print("Nenhum modelo foi treinado (nenhuma combinação de parâmetros foi melhor).")

Carregando dados...
Vetorizando textos...

Configurando Grid Search...

Iniciando Grid Search...

Testando: units1=128, units2=32, dropout=0.2, lr=0.001, opt=adam, batch=32, epochs=20, activation=relu, regularizer=None


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Recall (Negativo): 0.7391
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Recall (Negativo): 0.7857
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
Recall (Negativo): 0.9571
Recall (Negativo) médio (validação cruzada): 0.8273
------------------------------

Tempo total de execução: 0.15 minutos

Melhores parâmetros encontrados:
units1: 128
units2: 32
dropout_rate: 0.2
learning_rate: 0.001
optimizer: adam
batch_size: 32
epochs: 20
activation: relu
regularizer: None

Melhor score de validação: 0.8273

Avaliando no conjunto de teste...
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

Acurácia no teste: 0.8095
F1-Score no teste: 0.8103

Relatório de Classificação:
              precision    recall  f1-score   support

    Negative       0.77      0.85      0.81        52
     Neutral       0.93      0.77      0.84        35
    Positive       0.79      0.80      0.79        60

    accuracy                           0.81       147
   macro avg       0.83      0.81      0.81       1

Melhores parâmetros encontrados:
units1: 128
units2: 32
dropout_rate: 0.2
learning_rate: 0.001
optimizer: adam
batch_size: 32
epochs: 20
activation: relu
regularizer: None

Melhor score de validação: 0.8130

In [ ]:
from model2vec import StaticModel
from typing import List, Tuple, Dict

from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

import pandas as pd
import numpy as np

class Model2VecEmbeddings():
    """Wrapper para o Model2Vec como Embeddings do LangChain"""
    def __init__(self, model_name: str = "minishlab/potion-base-2M", similarity_threshold: float = 0.85):
        self.model = StaticModel.from_pretrained(model_name)
        self.similarity_threshold = similarity_threshold

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode(texts).tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.model.encode([text]).tolist()[0]

In [ ]:
from model2vec import StaticModel
from typing import List
import numpy as np

class Model2VecEmbeddings():
    """Wrapper para o Model2Vec como Embeddings do LangChain"""
    def __init__(self, model_name: str = "minishlab/potion-base-2M", similarity_threshold: float = 0.85):
        self.model = StaticModel.from_pretrained(model_name)
        self.similarity_threshold = similarity_threshold

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode(texts).tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.model.encode([text]).tolist()[0]

def get_model2vec_embedding(text: str) -> np.ndarray:
    """
    Gera o embedding de uma única frase usando Model2VecEmbeddings e retorna um array NumPy.

    Args:
        text: A frase para a qual gerar o embedding.

    Returns:
        Um array NumPy representando o vetor de embedding da frase.
    """
    emb = Model2VecEmbeddings()
    vector_list = emb.embed_query(text=text)  # Obtém o embedding como uma lista
    vector_np = np.array(vector_list)       # Converte a lista para um array NumPy
    return vector_np

# Testando a função modificada
frase_teste_amor = "I love it!"
embedding_amor = get_model2vec_embedding(frase_teste_amor)
print(f"Len Vetor da frase {len(embedding_amor)}")
#print(f"Dimensão do embedding: {embedding_amor.shape[0]}") # Correção aqui


def get_mean_embedding(text):
    doc = nlp(text)
    return doc.vector

frase_teste_amor = "I love it!"
embedding_amor = get_mean_embedding(frase_teste_amor)
print(f"Len Vetor da frase {len(embedding_amor)}")

Len Vetor da frase '64
Len Vetor da frase '300


In [ ]:
def get_mean_embedding(text):
    emb = Model2VecEmbeddings()
    vector_list = emb.embed_query(text=text)  # Obtém o embedding como uma lista
    vector_np = np.array(vector_list)       # Converte a lista para um array NumPy
    return vector_np

# Testando a função com a frase "I love it!"
frase_teste = "I love it!"
embedding_frase = get_mean_embedding(frase_teste)

print(f"Embedding da frase '{frase_teste}':")
print(embedding_frase)
print(f"Dimensão do embedding: {len(embedding_frase[0])}")

Embedding da frase 'I love it!':
[-2.36115545e-01 -3.12402427e-01 -2.22327933e-01  1.72482193e-01
 -2.72862583e-01  5.01802981e-01 -1.04314387e-01  3.64558734e-02
  1.62822381e-02  1.24722440e-02 -1.18248677e-02 -8.00213441e-02
  1.17085604e-02  1.73785780e-02  9.82469916e-02 -4.03998718e-02
  4.83643003e-02 -8.57188106e-02  6.39445335e-03  4.36404236e-02
  3.37613001e-02 -1.24388322e-01  1.70758590e-01 -1.58254161e-01
  1.04872413e-01  2.28477959e-02  2.67530195e-02  2.31581349e-02
  2.92013716e-02  6.63197786e-02 -2.02581927e-01 -1.36161223e-01
  6.31300062e-02 -1.55854970e-01  1.80398691e-02 -1.32623091e-01
  4.46904600e-02 -7.34133506e-03 -7.21999854e-02  1.70209974e-01
 -4.70680417e-04  2.47025378e-02  5.52542582e-02 -8.42899159e-02
  2.42101066e-02  3.58358286e-02  7.77482986e-02 -3.91708724e-02
  6.36928827e-02  7.14799315e-02 -1.22383609e-01  2.86588389e-02
  2.41834428e-02 -9.13440660e-02  3.47915515e-02  4.10464928e-02
 -9.48220417e-02 -5.92718879e-03  1.13304265e-01  1.96934

TypeError: object of type 'numpy.float64' has no len()

In [ ]:
nlp.shape[1]

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold  # Adicionado StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score
from sklearn.preprocessing import LabelEncoder  # Importado LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizers import l2
import time
from sklearn.metrics import make_scorer #importa o make_scorer
from sklearn.model_selection import cross_val_score

import spacy

# 1. Carregar e preparar os dados
print("Carregando dados...")
df_train = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/dataset_train_trim_synthetic_balanced.parquet')

if 'target' not in df_train.columns:
    df_train['target'] = df_train['sentiment']

# Codificar labels
sentiment_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
y = df_train['target'].map(sentiment_mapping)
X = df_train['comment_cleaned']

# Split dos dados
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

print("Dados Carregados.")

Carregando dados...
Dados Carregados.


# model_name="minishlab/potion-base-2M"

In [ ]:
# 2. Vetorização do texto
print("Vetorizando textos...")
# vectorizer = TfidfVectorizer(
#     max_features=8000,
#     ngram_range=(1, 2),
#     stop_words='english'
# )
# X_train_vec = vectorizer.fit_transform(X_train)
# X_test_vec = vectorizer.transform(X_test)

# # Converter para arrays densos
# X_train_vec = X_train_vec.toarray()
# X_test_vec = X_test_vec.toarray()

embedding_model_instance = Model2VecEmbeddings(model_name="minishlab/potion-base-2M")

def get_mean_embedding(text):
    """
    Generates embedding for a single text using the pre-loaded Model2VecEmbeddings instance.
    Note: This function now uses the globally instantiated 'embedding_model_instance'.
    The name 'get_mean_embedding' might not be strictly accurate for all models,
    as it's now getting a single vector per text, not necessarily a mean of token vectors.
    """
    # Usa a instância do Model2VecEmbeddings e chama o método para um único texto
    # embed_query retorna uma lista de floats [float, float, ...]
    return embedding_model_instance.embed_query(text)

# # Carregar um modelo do SpaCy (escolha um que inclua embeddings)
# nlp = spacy.load("en_core_web_lg")  # ou "en_core_web_lg" para embeddings maiores

# def get_mean_embedding(text):
#     doc = nlp(text)
#     return doc.vector

X_train_vec = np.array([get_mean_embedding(text) for text in X_train])
X_test_vec = np.array([get_mean_embedding(text) for text in X_test])

print("out")

# 3. Função para criar modelo
def create_model(units1=64, units2=32, dropout_rate=0.3,
                learning_rate=0.001, optimizer='adam',
                activation='relu', regularizer=None):
    model = Sequential()
    # Exemplo de matriz de custos (penalizando mais falsos positivos para a classe 0 - Negativo)
    class_weights = {0: 5, 1: 1, 2: 1} # Peso 5 para a classe Negativo, 1 para as outras
    # Camada de entrada
    model.add(Dense(
        units=units1,
        input_dim=X_train_vec.shape[1],
        activation=activation,
        kernel_regularizer=regularizer
    ))
    model.add(Dropout(dropout_rate))

    # Camada oculta
    model.add(Dense(
        units=units2,
        activation=activation,
        kernel_regularizer=regularizer
    ))
    model.add(Dropout(dropout_rate))

    # Camada de saída
    model.add(Dense(3, activation='softmax'))

    # Compilar modelo
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    else:
        opt = RMSprop(learning_rate=learning_rate)

    model.compile(
        loss='sparse_categorical_crossentropy',  # Importante: para labels inteiros
        optimizer=opt,
        metrics=['accuracy']
    )

    return model

#Testando: units1=128, units2=32, dropout=0.2, lr=0.001, opt=adam, batch=32, epochs=20, activation=relu, regularizer=None

# 4. Configuração da Grid Search
print("\nConfigurando Grid Search...")
param_grid = {
    'units1': [128],
    'units2': [32],
    'dropout_rate': [0.2],
    'learning_rate': [0.001],
    'optimizer': ['adam'],
    'batch_size': [32],
    'epochs': [200],
    'activation': ['relu'],
    'regularizer': [None]
}

# 5. Grid Search Manual
print("\nIniciando Grid Search...")
start_time = time.time()

best_score = -1
best_params = {}
best_model = None

# Para cada combinação de parâmetros
for units1 in param_grid['units1']:
    for units2 in param_grid['units2']:
        for dropout_rate in param_grid['dropout_rate']:
            for learning_rate in param_grid['learning_rate']:
                for optimizer_name in param_grid['optimizer']:
                    for batch_size in param_grid['batch_size']:
                        for epochs in param_grid['epochs']:
                            for activation in param_grid['activation']:
                                for regularizer_type in param_grid['regularizer']:
                                    print(f"\nTestando: units1={units1}, units2={units2}, dropout={dropout_rate}, lr={learning_rate}, opt={optimizer_name}, batch={batch_size}, epochs={epochs}, activation={activation}, regularizer={regularizer_type}")

                                    # Criar o modelo com os parâmetros atuais
                                    model = create_model(
                                        units1=units1,
                                        units2=units2,
                                        dropout_rate=dropout_rate,
                                        learning_rate=learning_rate,
                                        optimizer=optimizer_name,
                                        activation=activation,
                                        regularizer=regularizer_type
                                    )

                                    # Validação cruzada (cross-validation) - StratifiedKFold para manter a proporção das classes
                                    kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)  # Mantém a proporção das classes
                                    scores_acc = []
                                    scores_recall_neg = []

                                    for train_index, val_index in kfold.split(X_train_vec, y_train):
                                        X_train_fold, X_val_fold = X_train_vec[train_index], X_train_vec[val_index]
                                        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usando .iloc para indexar corretamente
                                        # Treinar o modelo no conjunto de treinamento da dobra (fold)
                                        model.fit(X_train_fold, y_train_fold, batch_size=batch_size, epochs=epochs, verbose=0) # verbose=0 silencia a saída de treinamento

                                        # Fazer a predição no conjunto de validação da dobra
                                        y_pred_fold = np.argmax(model.predict(X_val_fold), axis=-1)  # Convertendo as predições de probabilidade para rótulos de classe
                                        # Calcular o F1-score ponderado para a dobra
                                        # f1 = f1_score(y_val_fold, y_pred_fold, average='weighted')
                                        # scores.append(f1)
                                        overall_accurary = accuracy_score(y_val_fold, y_pred_fold)
                                        print(f"Overall Accuracy: {overall_accurary:.4f}")
                                        recall_negativo = recall_score(y_val_fold, y_pred_fold, average=None, labels=[0])[0]  # labels=[0] para o recall da classe 0 (Negativo)
                                        scores_acc.append(overall_accurary)

                                        print(f"Recall (Negativo): {recall_negativo:.4f}")
                                        scores_recall_neg.append(recall_negativo)

                                    # Calcular a média dos scores de validação cruzada
                                    # mean_f1 = np.mean(scores)
                                    # print(f"F1-score médio (validação cruzada): {mean_f1:.4f}")
                                    mean_accuracy = np.mean(scores_acc)
                                    print(f"Overall Accuracy médio (validação cruzada): {mean_accuracy:.4f}")

                                    mean_recall_negativo = np.mean(scores_recall_neg)
                                    print(f"Recall (Negativo) médio (validação cruzada): {mean_recall_negativo:.4f}")


                                    # Se o score for melhor, salvar os parâmetros e o modelo
                                    if mean_recall_negativo > best_score:
                                        best_score = mean_recall_negativo
                                        best_params = {
                                            'units1': units1,
                                            'units2': units2,
                                            'dropout_rate': dropout_rate,
                                            'learning_rate': learning_rate,
                                            'optimizer': optimizer_name,
                                            'batch_size': batch_size,
                                            'epochs': epochs,
                                            'activation': activation,
                                            'regularizer': regularizer_type
                                        }
                                        best_model = model # Salvando o modelo treinado com os melhores parâmetros
                                    print("-" * 30)  # Separador visual


print(f"\nTempo total de execução: {(time.time() - start_time) / 60:.2f} minutos")

# 7. Resultados
print("\nMelhores parâmetros encontrados:")
for param, value in best_params.items():
    print(f"{param}: {value}")

print(f"\nMelhor score de validação: {best_score:.4f}")

# 8. Avaliação no conjunto de teste (usando o melhor modelo treinado)
print("\nAvaliando no conjunto de teste...")

if best_model is not None:  # Certifique-se de que um modelo foi treinado
    y_pred = np.argmax(best_model.predict(X_test_vec), axis=-1) # Convertendo as predições de probabilidade para rótulos de classe
    test_accuracy = accuracy_score(y_test, y_pred)
    test_f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"\nAcurácia no teste: {test_accuracy:.4f}")
    print(f"F1-Score no teste: {test_f1:.4f}")

    # 9. Relatório detalhado
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred, target_names=sentiment_mapping.keys()))

    # 10. Análise por classe (especialmente para negativas)
    print("\nMétricas para classe Negative:")
    neg_mask = (y_test == 0)
    if np.any(neg_mask):  # Garante que haja exemplos da classe negativa
        neg_accuracy = accuracy_score(y_test[neg_mask], y_pred[neg_mask])
        neg_f1 = f1_score(y_test[neg_mask], y_pred[neg_mask], average='weighted')

        print(f"Acurácia para Negative: {neg_accuracy:.4f}")
        print(f"F1-Score para Negative: {neg_f1:.4f}")
    else:
        print("Nenhum exemplo da classe Negative encontrado no conjunto de teste.")

    # 11. Exemplos de erros (falsos negativos)
    false_negatives = X_test[ (y_test == 0) & (y_pred != 0) ]  # Corrigido para usar os índices originais
    if len(false_negatives) > 0:
        print("\nExemplos de falsos negativos:")
        #Recupera o texto original usando os indices originais.
        false_negatives_indices = np.where((y_test == 0) & (y_pred != 0))[0] # Obtem os indices dos falsos negativos.
        for i, index in enumerate(false_negatives_indices[:5]): # Usa os indices para acessar X_test
            print(f"{i+1}. {X.iloc[index]}") # Usa X.iloc para pegar os textos originais.
    else:
        print("Nenhum falso negativo encontrado.")
else:
    print("Nenhum modelo foi treinado (nenhuma combinação de parâmetros foi melhor).")

# New Data Validation

# 1. Carregar o novo dataset
new_data_path = '/content/drive/MyDrive/Colab Notebooks/dataset_valid_with_sentiment_fix_negative.parquet'
print(f"\n--- Avaliando no dataset: {new_data_path} ---")

try:
    new_data = pd.read_parquet(new_data_path)
    print(f"Dataset '{new_data_path}' carregado com sucesso. Shape: {new_data.shape}")

    # 2. Separar textos (X_new) e rótulos (y_new)
    # Assuma que as colunas de texto e rótulo são as mesmas do dataset de treino.
    # Ajuste 'text_column' e 'sentiment_column' conforme necessário.
    text_column = 'comment_cleaned'       # Nome da coluna de texto no seu parquet
    sentiment_column = 'sentiment' # Nome da coluna de sentimento no seu parquet
    # Codificar labels
    sentiment_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
    # y = df_train['target'].map(sentiment_mapping)
    # X = df_train['comment_cleaned']
    if text_column not in new_data.columns:
        print(f"Erro: Coluna de texto '{text_column}' não encontrada no dataset.")
    elif sentiment_column not in new_data.columns:
        print(f"Erro: Coluna de sentimento '{sentiment_column}' não encontrada no dataset.")
    else:
        X_new = new_data[text_column]
        y_new = new_data[sentiment_column].map(sentiment_mapping)

        # Certifique-se de que y_new está no formato numérico esperado (inteiros)
        # Pode ser necessário aplicar o mesmo mapeamento que foi usado no treino
        # Exemplo (se sentiment_mapping mapeia strings para ints):
        # y_new = y_new.map({v: k for k, v in sentiment_mapping.items()})
        # Certifique-se de que os rótulos numéricos correspondem aos usados no treino.
        # Se y_new já vier como inteiros 0, 1, 2, essa linha acima não é necessária.
        print(f"Textos extraídos ({len(X_new)}). Rótulos extraídos ({len(y_new)}).")
        print("Primeiros 5 rótulos do novo dataset:", y_new.head().tolist())


        # 3. Gerar embeddings para o novo dataset
        print("Gerando embeddings para o novo dataset...")

        # É crucial usar a mesma instância do embedder já carregado
        # A sua classe Model2VecEmbeddings tem o método embed_documents para listas,
        # que é mais eficiente do que chamar embed_query em loop.
        if 'embedding_model_instance' in globals() and embedding_model_instance is not None:
             X_new_vec_list = embedding_model_instance.embed_documents(X_new.tolist())
             X_new_vec = np.array(X_new_vec_list)
             print(f"Embeddings para o novo dataset gerados: {X_new_vec.shape}")

             # 4. Avaliar o best_model no novo conjunto de embeddings
             print("\nAvaliando o melhor modelo no novo dataset...")

             if 'best_model' in globals() and best_model is not None:
                 # Fazer as predições usando o modelo treinado
                 y_pred_new_proba = best_model.predict(X_new_vec)
                 y_pred_new = np.argmax(y_pred_new_proba, axis=-1) # Converter predições para rótulos

                 # 5. Exibir métricas de avaliação
                 print("\nRelatório de Avaliação no Novo Dataset:")
                 # Use y_new e y_pred_new
                 test_accuracy_new = accuracy_score(y_new, y_pred_new)
                 test_f1_new = f1_score(y_new, y_pred_new, average='weighted')
                 recall_negativo_new = recall_score(y_new, y_pred_new, average=None, labels=[0])[0] # Recall para a classe 0 (Negativo)

                 print(f"Acurácia no novo dataset: {test_accuracy_new:.4f}")
                 print(f"F1-Score no novo dataset: {test_f1_new:.4f}")
                 print(f"Recall para classe 'Negative' (0) no novo dataset: {recall_negativo_new:.4f}")

                 print("\nClassification Report no Novo Dataset:")
                 # Assegure-se de que sentiment_mapping está definido e correto para target_names
                 if 'sentiment_mapping' in globals():
                     target_names_list = [str(sentiment_mapping[i]) for i in sorted(sentiment_mapping.keys())]
                     print(classification_report(y_new, y_pred_new, target_names=target_names_list))
                 else:
                      print(classification_report(y_new, y_pred_new)) # Sem nomes se mapping não disponível

                 # Opcional: Análise de Falsos Negativos no NOVO dataset
                 print("\nAnálise de Falsos Negativos no Novo Dataset:")
                 # Encontrar os índices onde o rótulo real é 0 (Negative) mas a predição não é 0
                 false_negatives_indices_new = new_data.index[(y_new == 0) & (y_pred_new != 0)]

                 if len(false_negatives_indices_new) > 0:
                     print(f"Encontrados {len(false_negatives_indices_new)} falsos negativos no novo dataset.")
                     print("Exemplos de falsos negativos (primeiros 5):")
                     # Use .loc com os índices do dataframe original para pegar os textos e rótulos
                     for i, idx in enumerate(false_negatives_indices_new[:5]):
                         original_text = new_data.loc[idx, text_column]
                         true_label = y_new.loc[idx]
                         predicted_label = y_pred_new[new_data.index.get_loc(idx)] # Encontra a predição pelo índice original
                         # Mapear rótulos numéricos para nomes se sentiment_mapping estiver disponível
                         true_label_name = sentiment_mapping.get(true_label, str(true_label)) if 'sentiment_mapping' in globals() else str(true_label)
                         predicted_label_name = sentiment_mapping.get(predicted_label, str(predicted_label)) if 'sentiment_mapping' in globals() else str(predicted_label)

                         print(f"  {i+1}. Texto: '{original_text}'")
                         print(f"     Real: {true_label_name} ({true_label}), Predito: {predicted_label_name} ({predicted_label})")
                 else:
                     print("Nenhum falso negativo encontrado no novo dataset.")

             else:
                 print("Erro: A variável 'best_model' não foi encontrada ou é None. Certifique-se de que o treinamento e a seleção do modelo foram executados antes da avaliação.")

except FileNotFoundError:
    print(f"Erro: Arquivo '{new_data_path}' não encontrado.")
except Exception as e:
    print(f"Ocorreu um erro durante o processamento do novo dataset: {e}")

print("\n--- Fim da avaliação no novo dataset ---")

Vetorizando textos...
out

Configurando Grid Search...

Iniciando Grid Search...

Testando: units1=128, units2=32, dropout=0.2, lr=0.001, opt=adam, batch=32, epochs=200, activation=relu, regularizer=None


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Overall Accuracy: 0.6327
Recall (Negativo): 0.6812
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Overall Accuracy: 0.9436
Recall (Negativo): 0.9429
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
Overall Accuracy: 1.0000
Recall (Negativo): 1.0000
Overall Accuracy médio (validação cruzada): 0.8587
Recall (Negativo) médio (validação cruzada): 0.8747
------------------------------

Tempo total de execução: 1.11 minutos

Melhores parâmetros encontrados:
units1: 128
units2: 32
dropout_rate: 0.2
learning_rate: 0.001
optimizer: adam
batch_size: 32
epochs: 200
activation: relu
regularizer: None

Melhor score de validação: 0.8747

Avaliando no conjunto de teste...
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 

Acurácia no teste: 0.7007
F1-Score no teste: 0.7010

Relatório de Classificação:
              precision    recall  f1-score   support

    Negative       0.76      0.65      0.70        52
     Neutral       0.63      0.74      0.68        35
    Positive       0.70      0.72

# model_name="minishlab/potion-base-4M"

In [ ]:
# 2. Vetorização do texto
print("Vetorizando textos...")
# vectorizer = TfidfVectorizer(
#     max_features=8000,
#     ngram_range=(1, 2),
#     stop_words='english'
# )
# X_train_vec = vectorizer.fit_transform(X_train)
# X_test_vec = vectorizer.transform(X_test)

# # Converter para arrays densos
# X_train_vec = X_train_vec.toarray()
# X_test_vec = X_test_vec.toarray()

embedding_model_instance = Model2VecEmbeddings(model_name="minishlab/potion-base-4M")

def get_mean_embedding(text):
    """
    Generates embedding for a single text using the pre-loaded Model2VecEmbeddings instance.
    Note: This function now uses the globally instantiated 'embedding_model_instance'.
    The name 'get_mean_embedding' might not be strictly accurate for all models,
    as it's now getting a single vector per text, not necessarily a mean of token vectors.
    """
    # Usa a instância do Model2VecEmbeddings e chama o método para um único texto
    # embed_query retorna uma lista de floats [float, float, ...]
    return embedding_model_instance.embed_query(text)

# # Carregar um modelo do SpaCy (escolha um que inclua embeddings)
# nlp = spacy.load("en_core_web_lg")  # ou "en_core_web_lg" para embeddings maiores

# def get_mean_embedding(text):
#     doc = nlp(text)
#     return doc.vector

X_train_vec = np.array([get_mean_embedding(text) for text in X_train])
X_test_vec = np.array([get_mean_embedding(text) for text in X_test])

print("out")

# 3. Função para criar modelo
def create_model(units1=64, units2=32, dropout_rate=0.3,
                learning_rate=0.001, optimizer='adam',
                activation='relu', regularizer=None):
    model = Sequential()
    # Exemplo de matriz de custos (penalizando mais falsos positivos para a classe 0 - Negativo)
    class_weights = {0: 5, 1: 1, 2: 1} # Peso 5 para a classe Negativo, 1 para as outras
    # Camada de entrada
    model.add(Dense(
        units=units1,
        input_dim=X_train_vec.shape[1],
        activation=activation,
        kernel_regularizer=regularizer
    ))
    model.add(Dropout(dropout_rate))

    # Camada oculta
    model.add(Dense(
        units=units2,
        activation=activation,
        kernel_regularizer=regularizer
    ))
    model.add(Dropout(dropout_rate))

    # Camada de saída
    model.add(Dense(3, activation='softmax'))

    # Compilar modelo
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    else:
        opt = RMSprop(learning_rate=learning_rate)

    model.compile(
        loss='sparse_categorical_crossentropy',  # Importante: para labels inteiros
        optimizer=opt,
        metrics=['accuracy']
    )

    return model

#Testando: units1=128, units2=32, dropout=0.2, lr=0.001, opt=adam, batch=32, epochs=20, activation=relu, regularizer=None

# 4. Configuração da Grid Search
print("\nConfigurando Grid Search...")
param_grid = {
    'units1': [128],
    'units2': [32],
    'dropout_rate': [0.2],
    'learning_rate': [0.001],
    'optimizer': ['adam'],
    'batch_size': [32],
    'epochs': [200],
    'activation': ['relu'],
    'regularizer': [None]
}

# 5. Grid Search Manual
print("\nIniciando Grid Search...")
start_time = time.time()

best_score = -1
best_params = {}
best_model = None

# Para cada combinação de parâmetros
for units1 in param_grid['units1']:
    for units2 in param_grid['units2']:
        for dropout_rate in param_grid['dropout_rate']:
            for learning_rate in param_grid['learning_rate']:
                for optimizer_name in param_grid['optimizer']:
                    for batch_size in param_grid['batch_size']:
                        for epochs in param_grid['epochs']:
                            for activation in param_grid['activation']:
                                for regularizer_type in param_grid['regularizer']:
                                    print(f"\nTestando: units1={units1}, units2={units2}, dropout={dropout_rate}, lr={learning_rate}, opt={optimizer_name}, batch={batch_size}, epochs={epochs}, activation={activation}, regularizer={regularizer_type}")

                                    # Criar o modelo com os parâmetros atuais
                                    model = create_model(
                                        units1=units1,
                                        units2=units2,
                                        dropout_rate=dropout_rate,
                                        learning_rate=learning_rate,
                                        optimizer=optimizer_name,
                                        activation=activation,
                                        regularizer=regularizer_type
                                    )

                                    # Validação cruzada (cross-validation) - StratifiedKFold para manter a proporção das classes
                                    kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)  # Mantém a proporção das classes
                                    scores_acc = []
                                    scores_recall_neg = []

                                    for train_index, val_index in kfold.split(X_train_vec, y_train):
                                        X_train_fold, X_val_fold = X_train_vec[train_index], X_train_vec[val_index]
                                        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usando .iloc para indexar corretamente
                                        # Treinar o modelo no conjunto de treinamento da dobra (fold)
                                        model.fit(X_train_fold, y_train_fold, batch_size=batch_size, epochs=epochs, verbose=0) # verbose=0 silencia a saída de treinamento

                                        # Fazer a predição no conjunto de validação da dobra
                                        y_pred_fold = np.argmax(model.predict(X_val_fold), axis=-1)  # Convertendo as predições de probabilidade para rótulos de classe
                                        # Calcular o F1-score ponderado para a dobra
                                        # f1 = f1_score(y_val_fold, y_pred_fold, average='weighted')
                                        # scores.append(f1)
                                        overall_accurary = accuracy_score(y_val_fold, y_pred_fold)
                                        print(f"Overall Accuracy: {overall_accurary:.4f}")
                                        recall_negativo = recall_score(y_val_fold, y_pred_fold, average=None, labels=[0])[0]  # labels=[0] para o recall da classe 0 (Negativo)
                                        scores_acc.append(overall_accurary)

                                        print(f"Recall (Negativo): {recall_negativo:.4f}")
                                        scores_recall_neg.append(recall_negativo)

                                    # Calcular a média dos scores de validação cruzada
                                    # mean_f1 = np.mean(scores)
                                    # print(f"F1-score médio (validação cruzada): {mean_f1:.4f}")
                                    mean_accuracy = np.mean(scores_acc)
                                    print(f"Overall Accuracy médio (validação cruzada): {mean_accuracy:.4f}")

                                    mean_recall_negativo = np.mean(scores_recall_neg)
                                    print(f"Recall (Negativo) médio (validação cruzada): {mean_recall_negativo:.4f}")


                                    # Se o score for melhor, salvar os parâmetros e o modelo
                                    if mean_recall_negativo > best_score:
                                        best_score = mean_recall_negativo
                                        best_params = {
                                            'units1': units1,
                                            'units2': units2,
                                            'dropout_rate': dropout_rate,
                                            'learning_rate': learning_rate,
                                            'optimizer': optimizer_name,
                                            'batch_size': batch_size,
                                            'epochs': epochs,
                                            'activation': activation,
                                            'regularizer': regularizer_type
                                        }
                                        best_model = model # Salvando o modelo treinado com os melhores parâmetros
                                    print("-" * 30)  # Separador visual


print(f"\nTempo total de execução: {(time.time() - start_time) / 60:.2f} minutos")

# 7. Resultados
print("\nMelhores parâmetros encontrados:")
for param, value in best_params.items():
    print(f"{param}: {value}")

print(f"\nMelhor score de validação: {best_score:.4f}")

# 8. Avaliação no conjunto de teste (usando o melhor modelo treinado)
print("\nAvaliando no conjunto de teste...")

if best_model is not None:  # Certifique-se de que um modelo foi treinado
    y_pred = np.argmax(best_model.predict(X_test_vec), axis=-1) # Convertendo as predições de probabilidade para rótulos de classe
    test_accuracy = accuracy_score(y_test, y_pred)
    test_f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"\nAcurácia no teste: {test_accuracy:.4f}")
    print(f"F1-Score no teste: {test_f1:.4f}")

    # 9. Relatório detalhado
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred, target_names=sentiment_mapping.keys()))

    # 10. Análise por classe (especialmente para negativas)
    print("\nMétricas para classe Negative:")
    neg_mask = (y_test == 0)
    if np.any(neg_mask):  # Garante que haja exemplos da classe negativa
        neg_accuracy = accuracy_score(y_test[neg_mask], y_pred[neg_mask])
        neg_f1 = f1_score(y_test[neg_mask], y_pred[neg_mask], average='weighted')

        print(f"Acurácia para Negative: {neg_accuracy:.4f}")
        print(f"F1-Score para Negative: {neg_f1:.4f}")
    else:
        print("Nenhum exemplo da classe Negative encontrado no conjunto de teste.")

    # 11. Exemplos de erros (falsos negativos)
    false_negatives = X_test[ (y_test == 0) & (y_pred != 0) ]  # Corrigido para usar os índices originais
    if len(false_negatives) > 0:
        print("\nExemplos de falsos negativos:")
        #Recupera o texto original usando os indices originais.
        false_negatives_indices = np.where((y_test == 0) & (y_pred != 0))[0] # Obtem os indices dos falsos negativos.
        for i, index in enumerate(false_negatives_indices[:5]): # Usa os indices para acessar X_test
            print(f"{i+1}. {X.iloc[index]}") # Usa X.iloc para pegar os textos originais.
    else:
        print("Nenhum falso negativo encontrado.")
else:
    print("Nenhum modelo foi treinado (nenhuma combinação de parâmetros foi melhor).")

# New Data Validation

# 1. Carregar o novo dataset
new_data_path = '/content/drive/MyDrive/Colab Notebooks/dataset_valid_with_sentiment_fix_negative.parquet'
print(f"\n--- Avaliando no dataset: {new_data_path} ---")

try:
    new_data = pd.read_parquet(new_data_path)
    print(f"Dataset '{new_data_path}' carregado com sucesso. Shape: {new_data.shape}")

    # 2. Separar textos (X_new) e rótulos (y_new)
    # Assuma que as colunas de texto e rótulo são as mesmas do dataset de treino.
    # Ajuste 'text_column' e 'sentiment_column' conforme necessário.
    text_column = 'comment_cleaned'       # Nome da coluna de texto no seu parquet
    sentiment_column = 'sentiment' # Nome da coluna de sentimento no seu parquet
    # Codificar labels
    sentiment_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
    # y = df_train['target'].map(sentiment_mapping)
    # X = df_train['comment_cleaned']
    if text_column not in new_data.columns:
        print(f"Erro: Coluna de texto '{text_column}' não encontrada no dataset.")
    elif sentiment_column not in new_data.columns:
        print(f"Erro: Coluna de sentimento '{sentiment_column}' não encontrada no dataset.")
    else:
        X_new = new_data[text_column]
        y_new = new_data[sentiment_column].map(sentiment_mapping)

        # Certifique-se de que y_new está no formato numérico esperado (inteiros)
        # Pode ser necessário aplicar o mesmo mapeamento que foi usado no treino
        # Exemplo (se sentiment_mapping mapeia strings para ints):
        # y_new = y_new.map({v: k for k, v in sentiment_mapping.items()})
        # Certifique-se de que os rótulos numéricos correspondem aos usados no treino.
        # Se y_new já vier como inteiros 0, 1, 2, essa linha acima não é necessária.
        print(f"Textos extraídos ({len(X_new)}). Rótulos extraídos ({len(y_new)}).")
        print("Primeiros 5 rótulos do novo dataset:", y_new.head().tolist())


        # 3. Gerar embeddings para o novo dataset
        print("Gerando embeddings para o novo dataset...")

        # É crucial usar a mesma instância do embedder já carregado
        # A sua classe Model2VecEmbeddings tem o método embed_documents para listas,
        # que é mais eficiente do que chamar embed_query em loop.
        if 'embedding_model_instance' in globals() and embedding_model_instance is not None:
             X_new_vec_list = embedding_model_instance.embed_documents(X_new.tolist())
             X_new_vec = np.array(X_new_vec_list)
             print(f"Embeddings para o novo dataset gerados: {X_new_vec.shape}")

             # 4. Avaliar o best_model no novo conjunto de embeddings
             print("\nAvaliando o melhor modelo no novo dataset...")

             if 'best_model' in globals() and best_model is not None:
                 # Fazer as predições usando o modelo treinado
                 y_pred_new_proba = best_model.predict(X_new_vec)
                 y_pred_new = np.argmax(y_pred_new_proba, axis=-1) # Converter predições para rótulos

                 # 5. Exibir métricas de avaliação
                 print("\nRelatório de Avaliação no Novo Dataset:")
                 # Use y_new e y_pred_new
                 test_accuracy_new = accuracy_score(y_new, y_pred_new)
                 test_f1_new = f1_score(y_new, y_pred_new, average='weighted')
                 recall_negativo_new = recall_score(y_new, y_pred_new, average=None, labels=[0])[0] # Recall para a classe 0 (Negativo)

                 print(f"Acurácia no novo dataset: {test_accuracy_new:.4f}")
                 print(f"F1-Score no novo dataset: {test_f1_new:.4f}")
                 print(f"Recall para classe 'Negative' (0) no novo dataset: {recall_negativo_new:.4f}")

                 print("\nClassification Report no Novo Dataset:")
                 # Assegure-se de que sentiment_mapping está definido e correto para target_names
                 if 'sentiment_mapping' in globals():
                     target_names_list = [str(sentiment_mapping[i]) for i in sorted(sentiment_mapping.keys())]
                     print(classification_report(y_new, y_pred_new, target_names=target_names_list))
                 else:
                      print(classification_report(y_new, y_pred_new)) # Sem nomes se mapping não disponível

                 # Opcional: Análise de Falsos Negativos no NOVO dataset
                 print("\nAnálise de Falsos Negativos no Novo Dataset:")
                 # Encontrar os índices onde o rótulo real é 0 (Negative) mas a predição não é 0
                 false_negatives_indices_new = new_data.index[(y_new == 0) & (y_pred_new != 0)]

                 if len(false_negatives_indices_new) > 0:
                     print(f"Encontrados {len(false_negatives_indices_new)} falsos negativos no novo dataset.")
                     print("Exemplos de falsos negativos (primeiros 5):")
                     # Use .loc com os índices do dataframe original para pegar os textos e rótulos
                     for i, idx in enumerate(false_negatives_indices_new[:5]):
                         original_text = new_data.loc[idx, text_column]
                         true_label = y_new.loc[idx]
                         predicted_label = y_pred_new[new_data.index.get_loc(idx)] # Encontra a predição pelo índice original
                         # Mapear rótulos numéricos para nomes se sentiment_mapping estiver disponível
                         true_label_name = sentiment_mapping.get(true_label, str(true_label)) if 'sentiment_mapping' in globals() else str(true_label)
                         predicted_label_name = sentiment_mapping.get(predicted_label, str(predicted_label)) if 'sentiment_mapping' in globals() else str(predicted_label)

                         print(f"  {i+1}. Texto: '{original_text}'")
                         print(f"     Real: {true_label_name} ({true_label}), Predito: {predicted_label_name} ({predicted_label})")
                 else:
                     print("Nenhum falso negativo encontrado no novo dataset.")

             else:
                 print("Erro: A variável 'best_model' não foi encontrada ou é None. Certifique-se de que o treinamento e a seleção do modelo foram executados antes da avaliação.")

except FileNotFoundError:
    print(f"Erro: Arquivo '{new_data_path}' não encontrado.")
except Exception as e:
    print(f"Ocorreu um erro durante o processamento do novo dataset: {e}")

print("\n--- Fim da avaliação no novo dataset ---")

Vetorizando textos...
out

Configurando Grid Search...

Iniciando Grid Search...

Testando: units1=128, units2=32, dropout=0.2, lr=0.001, opt=adam, batch=32, epochs=200, activation=relu, regularizer=None


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Overall Accuracy: 0.6480
Recall (Negativo): 0.6232
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Overall Accuracy: 0.9692
Recall (Negativo): 0.9857
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Overall Accuracy: 1.0000
Recall (Negativo): 1.0000
Overall Accuracy médio (validação cruzada): 0.8724
Recall (Negativo) médio (validação cruzada): 0.8696
------------------------------

Tempo total de execução: 1.14 minutos

Melhores parâmetros encontrados:
units1: 128
units2: 32
dropout_rate: 0.2
learning_rate: 0.001
optimizer: adam
batch_size: 32
epochs: 200
activation: relu
regularizer: None

Melhor score de validação: 0.8696

Avaliando no conjunto de teste...
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 

Acurácia no teste: 0.7551
F1-Score no teste: 0.7551

Relatório de Classificação:
              precision    recall  f1-score   support

    Negative       0.76      0.71      0.73        52
     Neutral       0.68      0.86      0.76        35
    Positive       0.81      0.73

# 8M

In [ ]:
# 2. Vetorização do texto
print("Vetorizando textos...")
# vectorizer = TfidfVectorizer(
#     max_features=8000,
#     ngram_range=(1, 2),
#     stop_words='english'
# )
# X_train_vec = vectorizer.fit_transform(X_train)
# X_test_vec = vectorizer.transform(X_test)

# # Converter para arrays densos
# X_train_vec = X_train_vec.toarray()
# X_test_vec = X_test_vec.toarray()

embedding_model_instance = Model2VecEmbeddings(model_name="minishlab/potion-base-8M")

def get_mean_embedding(text):
    """
    Generates embedding for a single text using the pre-loaded Model2VecEmbeddings instance.
    Note: This function now uses the globally instantiated 'embedding_model_instance'.
    The name 'get_mean_embedding' might not be strictly accurate for all models,
    as it's now getting a single vector per text, not necessarily a mean of token vectors.
    """
    # Usa a instância do Model2VecEmbeddings e chama o método para um único texto
    # embed_query retorna uma lista de floats [float, float, ...]
    return embedding_model_instance.embed_query(text)

# # Carregar um modelo do SpaCy (escolha um que inclua embeddings)
# nlp = spacy.load("en_core_web_lg")  # ou "en_core_web_lg" para embeddings maiores

# def get_mean_embedding(text):
#     doc = nlp(text)
#     return doc.vector

X_train_vec = np.array([get_mean_embedding(text) for text in X_train])
X_test_vec = np.array([get_mean_embedding(text) for text in X_test])

print("out")

# 3. Função para criar modelo
def create_model(units1=64, units2=32, dropout_rate=0.3,
                learning_rate=0.001, optimizer='adam',
                activation='relu', regularizer=None):
    model = Sequential()
    # Exemplo de matriz de custos (penalizando mais falsos positivos para a classe 0 - Negativo)
    class_weights = {0: 5, 1: 1, 2: 1} # Peso 5 para a classe Negativo, 1 para as outras
    # Camada de entrada
    model.add(Dense(
        units=units1,
        input_dim=X_train_vec.shape[1],
        activation=activation,
        kernel_regularizer=regularizer
    ))
    model.add(Dropout(dropout_rate))

    # Camada oculta
    model.add(Dense(
        units=units2,
        activation=activation,
        kernel_regularizer=regularizer
    ))
    model.add(Dropout(dropout_rate))

    # Camada de saída
    model.add(Dense(3, activation='softmax'))

    # Compilar modelo
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    else:
        opt = RMSprop(learning_rate=learning_rate)

    model.compile(
        loss='sparse_categorical_crossentropy',  # Importante: para labels inteiros
        optimizer=opt,
        metrics=['accuracy']
    )

    return model

#Testando: units1=128, units2=32, dropout=0.2, lr=0.001, opt=adam, batch=32, epochs=20, activation=relu, regularizer=None

# 4. Configuração da Grid Search
print("\nConfigurando Grid Search...")
param_grid = {
    'units1': [128],
    'units2': [32],
    'dropout_rate': [0.2],
    'learning_rate': [0.001],
    'optimizer': ['adam'],
    'batch_size': [32],
    'epochs': [200],
    'activation': ['relu'],
    'regularizer': [None]
}

# 5. Grid Search Manual
print("\nIniciando Grid Search...")
start_time = time.time()

best_score = -1
best_params = {}
best_model = None

# Para cada combinação de parâmetros
for units1 in param_grid['units1']:
    for units2 in param_grid['units2']:
        for dropout_rate in param_grid['dropout_rate']:
            for learning_rate in param_grid['learning_rate']:
                for optimizer_name in param_grid['optimizer']:
                    for batch_size in param_grid['batch_size']:
                        for epochs in param_grid['epochs']:
                            for activation in param_grid['activation']:
                                for regularizer_type in param_grid['regularizer']:
                                    print(f"\nTestando: units1={units1}, units2={units2}, dropout={dropout_rate}, lr={learning_rate}, opt={optimizer_name}, batch={batch_size}, epochs={epochs}, activation={activation}, regularizer={regularizer_type}")

                                    # Criar o modelo com os parâmetros atuais
                                    model = create_model(
                                        units1=units1,
                                        units2=units2,
                                        dropout_rate=dropout_rate,
                                        learning_rate=learning_rate,
                                        optimizer=optimizer_name,
                                        activation=activation,
                                        regularizer=regularizer_type
                                    )

                                    # Validação cruzada (cross-validation) - StratifiedKFold para manter a proporção das classes
                                    kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)  # Mantém a proporção das classes
                                    scores_acc = []
                                    scores_recall_neg = []

                                    for train_index, val_index in kfold.split(X_train_vec, y_train):
                                        X_train_fold, X_val_fold = X_train_vec[train_index], X_train_vec[val_index]
                                        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usando .iloc para indexar corretamente
                                        # Treinar o modelo no conjunto de treinamento da dobra (fold)
                                        model.fit(X_train_fold, y_train_fold, batch_size=batch_size, epochs=epochs, verbose=0) # verbose=0 silencia a saída de treinamento

                                        # Fazer a predição no conjunto de validação da dobra
                                        y_pred_fold = np.argmax(model.predict(X_val_fold), axis=-1)  # Convertendo as predições de probabilidade para rótulos de classe
                                        # Calcular o F1-score ponderado para a dobra
                                        # f1 = f1_score(y_val_fold, y_pred_fold, average='weighted')
                                        # scores.append(f1)
                                        overall_accurary = accuracy_score(y_val_fold, y_pred_fold)
                                        print(f"Overall Accuracy: {overall_accurary:.4f}")
                                        recall_negativo = recall_score(y_val_fold, y_pred_fold, average=None, labels=[0])[0]  # labels=[0] para o recall da classe 0 (Negativo)
                                        scores_acc.append(overall_accurary)

                                        print(f"Recall (Negativo): {recall_negativo:.4f}")
                                        scores_recall_neg.append(recall_negativo)

                                    # Calcular a média dos scores de validação cruzada
                                    # mean_f1 = np.mean(scores)
                                    # print(f"F1-score médio (validação cruzada): {mean_f1:.4f}")
                                    mean_accuracy = np.mean(scores_acc)
                                    print(f"Overall Accuracy médio (validação cruzada): {mean_accuracy:.4f}")

                                    mean_recall_negativo = np.mean(scores_recall_neg)
                                    print(f"Recall (Negativo) médio (validação cruzada): {mean_recall_negativo:.4f}")


                                    # Se o score for melhor, salvar os parâmetros e o modelo
                                    if mean_recall_negativo > best_score:
                                        best_score = mean_recall_negativo
                                        best_params = {
                                            'units1': units1,
                                            'units2': units2,
                                            'dropout_rate': dropout_rate,
                                            'learning_rate': learning_rate,
                                            'optimizer': optimizer_name,
                                            'batch_size': batch_size,
                                            'epochs': epochs,
                                            'activation': activation,
                                            'regularizer': regularizer_type
                                        }
                                        best_model = model # Salvando o modelo treinado com os melhores parâmetros
                                    print("-" * 30)  # Separador visual


print(f"\nTempo total de execução: {(time.time() - start_time) / 60:.2f} minutos")

# 7. Resultados
print("\nMelhores parâmetros encontrados:")
for param, value in best_params.items():
    print(f"{param}: {value}")

print(f"\nMelhor score de validação: {best_score:.4f}")

# 8. Avaliação no conjunto de teste (usando o melhor modelo treinado)
print("\nAvaliando no conjunto de teste...")

if best_model is not None:  # Certifique-se de que um modelo foi treinado
    y_pred = np.argmax(best_model.predict(X_test_vec), axis=-1) # Convertendo as predições de probabilidade para rótulos de classe
    test_accuracy = accuracy_score(y_test, y_pred)
    test_f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"\nAcurácia no teste: {test_accuracy:.4f}")
    print(f"F1-Score no teste: {test_f1:.4f}")

    # 9. Relatório detalhado
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred, target_names=sentiment_mapping.keys()))

    # 10. Análise por classe (especialmente para negativas)
    print("\nMétricas para classe Negative:")
    neg_mask = (y_test == 0)
    if np.any(neg_mask):  # Garante que haja exemplos da classe negativa
        neg_accuracy = accuracy_score(y_test[neg_mask], y_pred[neg_mask])
        neg_f1 = f1_score(y_test[neg_mask], y_pred[neg_mask], average='weighted')

        print(f"Acurácia para Negative: {neg_accuracy:.4f}")
        print(f"F1-Score para Negative: {neg_f1:.4f}")
    else:
        print("Nenhum exemplo da classe Negative encontrado no conjunto de teste.")

    # 11. Exemplos de erros (falsos negativos)
    false_negatives = X_test[ (y_test == 0) & (y_pred != 0) ]  # Corrigido para usar os índices originais
    if len(false_negatives) > 0:
        print("\nExemplos de falsos negativos:")
        #Recupera o texto original usando os indices originais.
        false_negatives_indices = np.where((y_test == 0) & (y_pred != 0))[0] # Obtem os indices dos falsos negativos.
        for i, index in enumerate(false_negatives_indices[:5]): # Usa os indices para acessar X_test
            print(f"{i+1}. {X.iloc[index]}") # Usa X.iloc para pegar os textos originais.
    else:
        print("Nenhum falso negativo encontrado.")
else:
    print("Nenhum modelo foi treinado (nenhuma combinação de parâmetros foi melhor).")

# New Data Validation

# 1. Carregar o novo dataset
new_data_path = '/content/drive/MyDrive/Colab Notebooks/dataset_valid_with_sentiment_fix_negative.parquet'
print(f"\n--- Avaliando no dataset: {new_data_path} ---")

try:
    new_data = pd.read_parquet(new_data_path)
    print(f"Dataset '{new_data_path}' carregado com sucesso. Shape: {new_data.shape}")

    # 2. Separar textos (X_new) e rótulos (y_new)
    # Assuma que as colunas de texto e rótulo são as mesmas do dataset de treino.
    # Ajuste 'text_column' e 'sentiment_column' conforme necessário.
    text_column = 'comment_cleaned'       # Nome da coluna de texto no seu parquet
    sentiment_column = 'sentiment' # Nome da coluna de sentimento no seu parquet
    # Codificar labels
    sentiment_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
    # y = df_train['target'].map(sentiment_mapping)
    # X = df_train['comment_cleaned']
    if text_column not in new_data.columns:
        print(f"Erro: Coluna de texto '{text_column}' não encontrada no dataset.")
    elif sentiment_column not in new_data.columns:
        print(f"Erro: Coluna de sentimento '{sentiment_column}' não encontrada no dataset.")
    else:
        X_new = new_data[text_column]
        y_new = new_data[sentiment_column].map(sentiment_mapping)

        # Certifique-se de que y_new está no formato numérico esperado (inteiros)
        # Pode ser necessário aplicar o mesmo mapeamento que foi usado no treino
        # Exemplo (se sentiment_mapping mapeia strings para ints):
        # y_new = y_new.map({v: k for k, v in sentiment_mapping.items()})
        # Certifique-se de que os rótulos numéricos correspondem aos usados no treino.
        # Se y_new já vier como inteiros 0, 1, 2, essa linha acima não é necessária.
        print(f"Textos extraídos ({len(X_new)}). Rótulos extraídos ({len(y_new)}).")
        print("Primeiros 5 rótulos do novo dataset:", y_new.head().tolist())


        # 3. Gerar embeddings para o novo dataset
        print("Gerando embeddings para o novo dataset...")

        # É crucial usar a mesma instância do embedder já carregado
        # A sua classe Model2VecEmbeddings tem o método embed_documents para listas,
        # que é mais eficiente do que chamar embed_query em loop.
        if 'embedding_model_instance' in globals() and embedding_model_instance is not None:
             X_new_vec_list = embedding_model_instance.embed_documents(X_new.tolist())
             X_new_vec = np.array(X_new_vec_list)
             print(f"Embeddings para o novo dataset gerados: {X_new_vec.shape}")

             # 4. Avaliar o best_model no novo conjunto de embeddings
             print("\nAvaliando o melhor modelo no novo dataset...")

             if 'best_model' in globals() and best_model is not None:
                 # Fazer as predições usando o modelo treinado
                 y_pred_new_proba = best_model.predict(X_new_vec)
                 y_pred_new = np.argmax(y_pred_new_proba, axis=-1) # Converter predições para rótulos

                 # 5. Exibir métricas de avaliação
                 print("\nRelatório de Avaliação no Novo Dataset:")
                 # Use y_new e y_pred_new
                 test_accuracy_new = accuracy_score(y_new, y_pred_new)
                 test_f1_new = f1_score(y_new, y_pred_new, average='weighted')
                 recall_negativo_new = recall_score(y_new, y_pred_new, average=None, labels=[0])[0] # Recall para a classe 0 (Negativo)

                 print(f"Acurácia no novo dataset: {test_accuracy_new:.4f}")
                 print(f"F1-Score no novo dataset: {test_f1_new:.4f}")
                 print(f"Recall para classe 'Negative' (0) no novo dataset: {recall_negativo_new:.4f}")

                 print("\nClassification Report no Novo Dataset:")
                 # Assegure-se de que sentiment_mapping está definido e correto para target_names
                 if 'sentiment_mapping' in globals():
                     target_names_list = [str(sentiment_mapping[i]) for i in sorted(sentiment_mapping.keys())]
                     print(classification_report(y_new, y_pred_new, target_names=target_names_list))
                 else:
                      print(classification_report(y_new, y_pred_new)) # Sem nomes se mapping não disponível

                 # Opcional: Análise de Falsos Negativos no NOVO dataset
                 print("\nAnálise de Falsos Negativos no Novo Dataset:")
                 # Encontrar os índices onde o rótulo real é 0 (Negative) mas a predição não é 0
                 false_negatives_indices_new = new_data.index[(y_new == 0) & (y_pred_new != 0)]

                 if len(false_negatives_indices_new) > 0:
                     print(f"Encontrados {len(false_negatives_indices_new)} falsos negativos no novo dataset.")
                     print("Exemplos de falsos negativos (primeiros 5):")
                     # Use .loc com os índices do dataframe original para pegar os textos e rótulos
                     for i, idx in enumerate(false_negatives_indices_new[:5]):
                         original_text = new_data.loc[idx, text_column]
                         true_label = y_new.loc[idx]
                         predicted_label = y_pred_new[new_data.index.get_loc(idx)] # Encontra a predição pelo índice original
                         # Mapear rótulos numéricos para nomes se sentiment_mapping estiver disponível
                         true_label_name = sentiment_mapping.get(true_label, str(true_label)) if 'sentiment_mapping' in globals() else str(true_label)
                         predicted_label_name = sentiment_mapping.get(predicted_label, str(predicted_label)) if 'sentiment_mapping' in globals() else str(predicted_label)

                         print(f"  {i+1}. Texto: '{original_text}'")
                         print(f"     Real: {true_label_name} ({true_label}), Predito: {predicted_label_name} ({predicted_label})")
                 else:
                     print("Nenhum falso negativo encontrado no novo dataset.")

             else:
                 print("Erro: A variável 'best_model' não foi encontrada ou é None. Certifique-se de que o treinamento e a seleção do modelo foram executados antes da avaliação.")

except FileNotFoundError:
    print(f"Erro: Arquivo '{new_data_path}' não encontrado.")
except Exception as e:
    print(f"Ocorreu um erro durante o processamento do novo dataset: {e}")

print("\n--- Fim da avaliação no novo dataset ---")

Vetorizando textos...
out

Configurando Grid Search...

Iniciando Grid Search...

Testando: units1=128, units2=32, dropout=0.2, lr=0.001, opt=adam, batch=32, epochs=200, activation=relu, regularizer=None


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Overall Accuracy: 0.6582
Recall (Negativo): 0.6522
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Overall Accuracy: 0.9641
Recall (Negativo): 0.9714
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Overall Accuracy: 1.0000
Recall (Negativo): 1.0000
Overall Accuracy médio (validação cruzada): 0.8741
Recall (Negativo) médio (validação cruzada): 0.8745
------------------------------

Tempo total de execução: 1.11 minutos

Melhores parâmetros encontrados:
units1: 128
units2: 32
dropout_rate: 0.2
learning_rate: 0.001
optimizer: adam
batch_size: 32
epochs: 200
activation: relu
regularizer: None

Melhor score de validação: 0.8745

Avaliando no conjunto de teste...
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 

Acurácia no teste: 0.7279
F1-Score no teste: 0.7277

Relatório de Classificação:
              precision    recall  f1-score   support

    Negative       0.76      0.67      0.71        52
     Neutral       0.67      0.80      0.73        35
    Positive       0.75      0.73

# 32M

In [ ]:
# 2. Vetorização do texto
print("Vetorizando textos...")
# vectorizer = TfidfVectorizer(
#     max_features=8000,
#     ngram_range=(1, 2),
#     stop_words='english'
# )
# X_train_vec = vectorizer.fit_transform(X_train)
# X_test_vec = vectorizer.transform(X_test)

# # Converter para arrays densos
# X_train_vec = X_train_vec.toarray()
# X_test_vec = X_test_vec.toarray()

embedding_model_instance = Model2VecEmbeddings(model_name="minishlab/potion-base-32M")

def get_mean_embedding(text):
    """
    Generates embedding for a single text using the pre-loaded Model2VecEmbeddings instance.
    Note: This function now uses the globally instantiated 'embedding_model_instance'.
    The name 'get_mean_embedding' might not be strictly accurate for all models,
    as it's now getting a single vector per text, not necessarily a mean of token vectors.
    """
    # Usa a instância do Model2VecEmbeddings e chama o método para um único texto
    # embed_query retorna uma lista de floats [float, float, ...]
    return embedding_model_instance.embed_query(text)

# # Carregar um modelo do SpaCy (escolha um que inclua embeddings)
# nlp = spacy.load("en_core_web_lg")  # ou "en_core_web_lg" para embeddings maiores

# def get_mean_embedding(text):
#     doc = nlp(text)
#     return doc.vector

X_train_vec = np.array([get_mean_embedding(text) for text in X_train])
X_test_vec = np.array([get_mean_embedding(text) for text in X_test])

print("out")

# 3. Função para criar modelo
def create_model(units1=64, units2=32, dropout_rate=0.3,
                learning_rate=0.001, optimizer='adam',
                activation='relu', regularizer=None):
    model = Sequential()
    # Exemplo de matriz de custos (penalizando mais falsos positivos para a classe 0 - Negativo)
    class_weights = {0: 5, 1: 1, 2: 1} # Peso 5 para a classe Negativo, 1 para as outras
    # Camada de entrada
    model.add(Dense(
        units=units1,
        input_dim=X_train_vec.shape[1],
        activation=activation,
        kernel_regularizer=regularizer
    ))
    model.add(Dropout(dropout_rate))

    # Camada oculta
    model.add(Dense(
        units=units2,
        activation=activation,
        kernel_regularizer=regularizer
    ))
    model.add(Dropout(dropout_rate))

    # Camada de saída
    model.add(Dense(3, activation='softmax'))

    # Compilar modelo
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    else:
        opt = RMSprop(learning_rate=learning_rate)

    model.compile(
        loss='sparse_categorical_crossentropy',  # Importante: para labels inteiros
        optimizer=opt,
        metrics=['accuracy']
    )

    return model

#Testando: units1=128, units2=32, dropout=0.2, lr=0.001, opt=adam, batch=32, epochs=20, activation=relu, regularizer=None

# 4. Configuração da Grid Search
print("\nConfigurando Grid Search...")
param_grid = {
    'units1': [128],
    'units2': [32],
    'dropout_rate': [0.2],
    'learning_rate': [0.001],
    'optimizer': ['adam'],
    'batch_size': [32],
    'epochs': [200],
    'activation': ['relu'],
    'regularizer': [None]
}

# 5. Grid Search Manual
print("\nIniciando Grid Search...")
start_time = time.time()

best_score = -1
best_params = {}
best_model = None

# Para cada combinação de parâmetros
for units1 in param_grid['units1']:
    for units2 in param_grid['units2']:
        for dropout_rate in param_grid['dropout_rate']:
            for learning_rate in param_grid['learning_rate']:
                for optimizer_name in param_grid['optimizer']:
                    for batch_size in param_grid['batch_size']:
                        for epochs in param_grid['epochs']:
                            for activation in param_grid['activation']:
                                for regularizer_type in param_grid['regularizer']:
                                    print(f"\nTestando: units1={units1}, units2={units2}, dropout={dropout_rate}, lr={learning_rate}, opt={optimizer_name}, batch={batch_size}, epochs={epochs}, activation={activation}, regularizer={regularizer_type}")

                                    # Criar o modelo com os parâmetros atuais
                                    model = create_model(
                                        units1=units1,
                                        units2=units2,
                                        dropout_rate=dropout_rate,
                                        learning_rate=learning_rate,
                                        optimizer=optimizer_name,
                                        activation=activation,
                                        regularizer=regularizer_type
                                    )

                                    # Validação cruzada (cross-validation) - StratifiedKFold para manter a proporção das classes
                                    kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)  # Mantém a proporção das classes
                                    scores_acc = []
                                    scores_recall_neg = []

                                    for train_index, val_index in kfold.split(X_train_vec, y_train):
                                        X_train_fold, X_val_fold = X_train_vec[train_index], X_train_vec[val_index]
                                        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usando .iloc para indexar corretamente
                                        # Treinar o modelo no conjunto de treinamento da dobra (fold)
                                        model.fit(X_train_fold, y_train_fold, batch_size=batch_size, epochs=epochs, verbose=0) # verbose=0 silencia a saída de treinamento

                                        # Fazer a predição no conjunto de validação da dobra
                                        y_pred_fold = np.argmax(model.predict(X_val_fold), axis=-1)  # Convertendo as predições de probabilidade para rótulos de classe
                                        # Calcular o F1-score ponderado para a dobra
                                        # f1 = f1_score(y_val_fold, y_pred_fold, average='weighted')
                                        # scores.append(f1)
                                        overall_accurary = accuracy_score(y_val_fold, y_pred_fold)
                                        print(f"Overall Accuracy: {overall_accurary:.4f}")
                                        recall_negativo = recall_score(y_val_fold, y_pred_fold, average=None, labels=[0])[0]  # labels=[0] para o recall da classe 0 (Negativo)
                                        scores_acc.append(overall_accurary)

                                        print(f"Recall (Negativo): {recall_negativo:.4f}")
                                        scores_recall_neg.append(recall_negativo)

                                    # Calcular a média dos scores de validação cruzada
                                    # mean_f1 = np.mean(scores)
                                    # print(f"F1-score médio (validação cruzada): {mean_f1:.4f}")
                                    mean_accuracy = np.mean(scores_acc)
                                    print(f"Overall Accuracy médio (validação cruzada): {mean_accuracy:.4f}")

                                    mean_recall_negativo = np.mean(scores_recall_neg)
                                    print(f"Recall (Negativo) médio (validação cruzada): {mean_recall_negativo:.4f}")


                                    # Se o score for melhor, salvar os parâmetros e o modelo
                                    if mean_recall_negativo > best_score:
                                        best_score = mean_recall_negativo
                                        best_params = {
                                            'units1': units1,
                                            'units2': units2,
                                            'dropout_rate': dropout_rate,
                                            'learning_rate': learning_rate,
                                            'optimizer': optimizer_name,
                                            'batch_size': batch_size,
                                            'epochs': epochs,
                                            'activation': activation,
                                            'regularizer': regularizer_type
                                        }
                                        best_model = model # Salvando o modelo treinado com os melhores parâmetros
                                    print("-" * 30)  # Separador visual


print(f"\nTempo total de execução: {(time.time() - start_time) / 60:.2f} minutos")

# 7. Resultados
print("\nMelhores parâmetros encontrados:")
for param, value in best_params.items():
    print(f"{param}: {value}")

print(f"\nMelhor score de validação: {best_score:.4f}")

# 8. Avaliação no conjunto de teste (usando o melhor modelo treinado)
print("\nAvaliando no conjunto de teste...")

if best_model is not None:  # Certifique-se de que um modelo foi treinado
    y_pred = np.argmax(best_model.predict(X_test_vec), axis=-1) # Convertendo as predições de probabilidade para rótulos de classe
    test_accuracy = accuracy_score(y_test, y_pred)
    test_f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"\nAcurácia no teste: {test_accuracy:.4f}")
    print(f"F1-Score no teste: {test_f1:.4f}")

    # 9. Relatório detalhado
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred, target_names=sentiment_mapping.keys()))

    # 10. Análise por classe (especialmente para negativas)
    print("\nMétricas para classe Negative:")
    neg_mask = (y_test == 0)
    if np.any(neg_mask):  # Garante que haja exemplos da classe negativa
        neg_accuracy = accuracy_score(y_test[neg_mask], y_pred[neg_mask])
        neg_f1 = f1_score(y_test[neg_mask], y_pred[neg_mask], average='weighted')

        print(f"Acurácia para Negative: {neg_accuracy:.4f}")
        print(f"F1-Score para Negative: {neg_f1:.4f}")
    else:
        print("Nenhum exemplo da classe Negative encontrado no conjunto de teste.")

    # 11. Exemplos de erros (falsos negativos)
    false_negatives = X_test[ (y_test == 0) & (y_pred != 0) ]  # Corrigido para usar os índices originais
    if len(false_negatives) > 0:
        print("\nExemplos de falsos negativos:")
        #Recupera o texto original usando os indices originais.
        false_negatives_indices = np.where((y_test == 0) & (y_pred != 0))[0] # Obtem os indices dos falsos negativos.
        for i, index in enumerate(false_negatives_indices[:5]): # Usa os indices para acessar X_test
            print(f"{i+1}. {X.iloc[index]}") # Usa X.iloc para pegar os textos originais.
    else:
        print("Nenhum falso negativo encontrado.")
else:
    print("Nenhum modelo foi treinado (nenhuma combinação de parâmetros foi melhor).")

# New Data Validation

# 1. Carregar o novo dataset
new_data_path = '/content/drive/MyDrive/Colab Notebooks/dataset_valid_with_sentiment_fix_negative.parquet'
print(f"\n--- Avaliando no dataset: {new_data_path} ---")

try:
    new_data = pd.read_parquet(new_data_path)
    print(f"Dataset '{new_data_path}' carregado com sucesso. Shape: {new_data.shape}")

    # 2. Separar textos (X_new) e rótulos (y_new)
    # Assuma que as colunas de texto e rótulo são as mesmas do dataset de treino.
    # Ajuste 'text_column' e 'sentiment_column' conforme necessário.
    text_column = 'comment_cleaned'       # Nome da coluna de texto no seu parquet
    sentiment_column = 'sentiment' # Nome da coluna de sentimento no seu parquet
    # Codificar labels
    sentiment_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
    # y = df_train['target'].map(sentiment_mapping)
    # X = df_train['comment_cleaned']
    if text_column not in new_data.columns:
        print(f"Erro: Coluna de texto '{text_column}' não encontrada no dataset.")
    elif sentiment_column not in new_data.columns:
        print(f"Erro: Coluna de sentimento '{sentiment_column}' não encontrada no dataset.")
    else:
        X_new = new_data[text_column]
        y_new = new_data[sentiment_column].map(sentiment_mapping)

        # Certifique-se de que y_new está no formato numérico esperado (inteiros)
        # Pode ser necessário aplicar o mesmo mapeamento que foi usado no treino
        # Exemplo (se sentiment_mapping mapeia strings para ints):
        # y_new = y_new.map({v: k for k, v in sentiment_mapping.items()})
        # Certifique-se de que os rótulos numéricos correspondem aos usados no treino.
        # Se y_new já vier como inteiros 0, 1, 2, essa linha acima não é necessária.
        print(f"Textos extraídos ({len(X_new)}). Rótulos extraídos ({len(y_new)}).")
        print("Primeiros 5 rótulos do novo dataset:", y_new.head().tolist())


        # 3. Gerar embeddings para o novo dataset
        print("Gerando embeddings para o novo dataset...")

        # É crucial usar a mesma instância do embedder já carregado
        # A sua classe Model2VecEmbeddings tem o método embed_documents para listas,
        # que é mais eficiente do que chamar embed_query em loop.
        if 'embedding_model_instance' in globals() and embedding_model_instance is not None:
             X_new_vec_list = embedding_model_instance.embed_documents(X_new.tolist())
             X_new_vec = np.array(X_new_vec_list)
             print(f"Embeddings para o novo dataset gerados: {X_new_vec.shape}")

             # 4. Avaliar o best_model no novo conjunto de embeddings
             print("\nAvaliando o melhor modelo no novo dataset...")

             if 'best_model' in globals() and best_model is not None:
                 # Fazer as predições usando o modelo treinado
                 y_pred_new_proba = best_model.predict(X_new_vec)
                 y_pred_new = np.argmax(y_pred_new_proba, axis=-1) # Converter predições para rótulos

                 # 5. Exibir métricas de avaliação
                 print("\nRelatório de Avaliação no Novo Dataset:")
                 # Use y_new e y_pred_new
                 test_accuracy_new = accuracy_score(y_new, y_pred_new)
                 test_f1_new = f1_score(y_new, y_pred_new, average='weighted')
                 recall_negativo_new = recall_score(y_new, y_pred_new, average=None, labels=[0])[0] # Recall para a classe 0 (Negativo)

                 print(f"Acurácia no novo dataset: {test_accuracy_new:.4f}")
                 print(f"F1-Score no novo dataset: {test_f1_new:.4f}")
                 print(f"Recall para classe 'Negative' (0) no novo dataset: {recall_negativo_new:.4f}")

                 print("\nClassification Report no Novo Dataset:")
                 # Assegure-se de que sentiment_mapping está definido e correto para target_names
                 if 'sentiment_mapping' in globals():
                     target_names_list = [str(sentiment_mapping[i]) for i in sorted(sentiment_mapping.keys())]
                     print(classification_report(y_new, y_pred_new, target_names=target_names_list))
                 else:
                      print(classification_report(y_new, y_pred_new)) # Sem nomes se mapping não disponível

                 # Opcional: Análise de Falsos Negativos no NOVO dataset
                 print("\nAnálise de Falsos Negativos no Novo Dataset:")
                 # Encontrar os índices onde o rótulo real é 0 (Negative) mas a predição não é 0
                 false_negatives_indices_new = new_data.index[(y_new == 0) & (y_pred_new != 0)]

                 if len(false_negatives_indices_new) > 0:
                     print(f"Encontrados {len(false_negatives_indices_new)} falsos negativos no novo dataset.")
                     print("Exemplos de falsos negativos (primeiros 5):")
                     # Use .loc com os índices do dataframe original para pegar os textos e rótulos
                     for i, idx in enumerate(false_negatives_indices_new[:5]):
                         original_text = new_data.loc[idx, text_column]
                         true_label = y_new.loc[idx]
                         predicted_label = y_pred_new[new_data.index.get_loc(idx)] # Encontra a predição pelo índice original
                         # Mapear rótulos numéricos para nomes se sentiment_mapping estiver disponível
                         true_label_name = sentiment_mapping.get(true_label, str(true_label)) if 'sentiment_mapping' in globals() else str(true_label)
                         predicted_label_name = sentiment_mapping.get(predicted_label, str(predicted_label)) if 'sentiment_mapping' in globals() else str(predicted_label)

                         print(f"  {i+1}. Texto: '{original_text}'")
                         print(f"     Real: {true_label_name} ({true_label}), Predito: {predicted_label_name} ({predicted_label})")
                 else:
                     print("Nenhum falso negativo encontrado no novo dataset.")

             else:
                 print("Erro: A variável 'best_model' não foi encontrada ou é None. Certifique-se de que o treinamento e a seleção do modelo foram executados antes da avaliação.")

except FileNotFoundError:
    print(f"Erro: Arquivo '{new_data_path}' não encontrado.")
except Exception as e:
    print(f"Ocorreu um erro durante o processamento do novo dataset: {e}")

print("\n--- Fim da avaliação no novo dataset ---")

Vetorizando textos...
out

Configurando Grid Search...

Iniciando Grid Search...

Testando: units1=128, units2=32, dropout=0.2, lr=0.001, opt=adam, batch=32, epochs=200, activation=relu, regularizer=None


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Overall Accuracy: 0.6837
Recall (Negativo): 0.6522
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
Overall Accuracy: 0.9795
Recall (Negativo): 0.9857
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Overall Accuracy: 1.0000
Recall (Negativo): 1.0000
Overall Accuracy médio (validação cruzada): 0.8877
Recall (Negativo) médio (validação cruzada): 0.8793
------------------------------

Tempo total de execução: 1.37 minutos

Melhores parâmetros encontrados:
units1: 128
units2: 32
dropout_rate: 0.2
learning_rate: 0.001
optimizer: adam
batch_size: 32
epochs: 200
activation: relu
regularizer: None

Melhor score de validação: 0.8793

Avaliando no conjunto de teste...
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 

Acurácia no teste: 0.7755
F1-Score no teste: 0.7717

Relatório de Classificação:
              precision    recall  f1-score   support

    Negative       0.79      0.65      0.72        52
     Neutral       0.76      0.74      0.75        35
    Positive       0.77      0.90

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold  # Adicionado StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score
from sklearn.preprocessing import LabelEncoder  # Importado LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizers import l2
import time
from sklearn.metrics import make_scorer #importa o make_scorer
from sklearn.model_selection import cross_val_score

import spacy

# 1. Carregar e preparar os dados
print("Carregando dados...")
df_train = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/dataset_train_trim_synthetic_balanced.parquet')

if 'target' not in df_train.columns:
    df_train['target'] = df_train['sentiment']

# Codificar labels
sentiment_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
y = df_train['target'].map(sentiment_mapping)
X = df_train['comment_cleaned']

# Split dos dados
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# 2. Vetorização do texto
print("Vetorizando textos...")
# vectorizer = TfidfVectorizer(
#     max_features=8000,
#     ngram_range=(1, 2),
#     stop_words='english'
# )
# X_train_vec = vectorizer.fit_transform(X_train)
# X_test_vec = vectorizer.transform(X_test)

# # Converter para arrays densos
# X_train_vec = X_train_vec.toarray()
# X_test_vec = X_test_vec.toarray()


# Carregar um modelo do SpaCy (escolha um que inclua embeddings)
nlp = spacy.load("en_core_web_lg")  # ou "en_core_web_lg" para embeddings maiores

def get_mean_embedding(text):
    doc = nlp(text)
    return doc.vector

X_train_vec = np.array([get_mean_embedding(text) for text in X_train])
X_test_vec = np.array([get_mean_embedding(text) for text in X_test])

# 3. Função para criar modelo
def create_model(units1=64, units2=32, dropout_rate=0.3,
                learning_rate=0.001, optimizer='adam',
                activation='relu', regularizer=None):
    model = Sequential()
    # Exemplo de matriz de custos (penalizando mais falsos positivos para a classe 0 - Negativo)
    class_weights = {0: 5, 1: 1, 2: 1} # Peso 5 para a classe Negativo, 1 para as outras
    # Camada de entrada
    model.add(Dense(
        units=units1,
        input_dim=X_train_vec.shape[1],
        activation=activation,
        kernel_regularizer=regularizer
    ))
    model.add(Dropout(dropout_rate))

    # Camada oculta
    model.add(Dense(
        units=units2,
        activation=activation,
        kernel_regularizer=regularizer
    ))
    model.add(Dropout(dropout_rate))

    # Camada de saída
    model.add(Dense(3, activation='softmax'))

    # Compilar modelo
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    else:
        opt = RMSprop(learning_rate=learning_rate)

    model.compile(
        loss='sparse_categorical_crossentropy',  # Importante: para labels inteiros
        optimizer=opt,
        metrics=['accuracy']
    )

    return model

#Testando: units1=128, units2=32, dropout=0.2, lr=0.001, opt=adam, batch=32, epochs=20, activation=relu, regularizer=None

# 4. Configuração da Grid Search
print("\nConfigurando Grid Search...")
param_grid = {
    'units1': [128],
    'units2': [32],
    'dropout_rate': [0.2],
    'learning_rate': [0.001],
    'optimizer': ['adam'],
    'batch_size': [32],
    'epochs': [20],
    'activation': ['relu'],
    'regularizer': [None]
}

# 5. Grid Search Manual
print("\nIniciando Grid Search...")
start_time = time.time()

best_score = -1
best_params = {}
best_model = None

# Para cada combinação de parâmetros
for units1 in param_grid['units1']:
    for units2 in param_grid['units2']:
        for dropout_rate in param_grid['dropout_rate']:
            for learning_rate in param_grid['learning_rate']:
                for optimizer_name in param_grid['optimizer']:
                    for batch_size in param_grid['batch_size']:
                        for epochs in param_grid['epochs']:
                            for activation in param_grid['activation']:
                                for regularizer_type in param_grid['regularizer']:
                                    print(f"\nTestando: units1={units1}, units2={units2}, dropout={dropout_rate}, lr={learning_rate}, opt={optimizer_name}, batch={batch_size}, epochs={epochs}, activation={activation}, regularizer={regularizer_type}")

                                    # Criar o modelo com os parâmetros atuais
                                    model = create_model(
                                        units1=units1,
                                        units2=units2,
                                        dropout_rate=dropout_rate,
                                        learning_rate=learning_rate,
                                        optimizer=optimizer_name,
                                        activation=activation,
                                        regularizer=regularizer_type
                                    )

                                    # Validação cruzada (cross-validation) - StratifiedKFold para manter a proporção das classes
                                    kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)  # Mantém a proporção das classes
                                    scores = []
                                    for train_index, val_index in kfold.split(X_train_vec, y_train):
                                        X_train_fold, X_val_fold = X_train_vec[train_index], X_train_vec[val_index]
                                        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usando .iloc para indexar corretamente
                                        # Treinar o modelo no conjunto de treinamento da dobra (fold)
                                        model.fit(X_train_fold, y_train_fold, batch_size=batch_size, epochs=epochs, verbose=0) # verbose=0 silencia a saída de treinamento

                                        # Fazer a predição no conjunto de validação da dobra
                                        y_pred_fold = np.argmax(model.predict(X_val_fold), axis=-1)  # Convertendo as predições de probabilidade para rótulos de classe
                                        # Calcular o F1-score ponderado para a dobra
                                        # f1 = f1_score(y_val_fold, y_pred_fold, average='weighted')
                                        # scores.append(f1)

                                        recall_negativo = recall_score(y_val_fold, y_pred_fold, average=None, labels=[0])[0]  # labels=[0] para o recall da classe 0 (Negativo)
                                        print(f"Recall (Negativo): {recall_negativo:.4f}")
                                        scores.append(recall_negativo)

                                    # Calcular a média dos scores de validação cruzada
                                    # mean_f1 = np.mean(scores)
                                    # print(f"F1-score médio (validação cruzada): {mean_f1:.4f}")

                                    mean_recall_negativo = np.mean(scores)
                                    print(f"Recall (Negativo) médio (validação cruzada): {mean_recall_negativo:.4f}")


                                    # Se o score for melhor, salvar os parâmetros e o modelo
                                    if mean_recall_negativo > best_score:
                                        best_score = mean_recall_negativo
                                        best_params = {
                                            'units1': units1,
                                            'units2': units2,
                                            'dropout_rate': dropout_rate,
                                            'learning_rate': learning_rate,
                                            'optimizer': optimizer_name,
                                            'batch_size': batch_size,
                                            'epochs': epochs,
                                            'activation': activation,
                                            'regularizer': regularizer_type
                                        }
                                        best_model = model # Salvando o modelo treinado com os melhores parâmetros
                                    print("-" * 30)  # Separador visual


print(f"\nTempo total de execução: {(time.time() - start_time) / 60:.2f} minutos")

# 7. Resultados
print("\nMelhores parâmetros encontrados:")
for param, value in best_params.items():
    print(f"{param}: {value}")

print(f"\nMelhor score de validação: {best_score:.4f}")

# 8. Avaliação no conjunto de teste (usando o melhor modelo treinado)
print("\nAvaliando no conjunto de teste...")

if best_model is not None:  # Certifique-se de que um modelo foi treinado
    y_pred = np.argmax(best_model.predict(X_test_vec), axis=-1) # Convertendo as predições de probabilidade para rótulos de classe
    test_accuracy = accuracy_score(y_test, y_pred)
    test_f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"\nAcurácia no teste: {test_accuracy:.4f}")
    print(f"F1-Score no teste: {test_f1:.4f}")

    # 9. Relatório detalhado
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred, target_names=sentiment_mapping.keys()))

    # 10. Análise por classe (especialmente para negativas)
    print("\nMétricas para classe Negative:")
    neg_mask = (y_test == 0)
    if np.any(neg_mask):  # Garante que haja exemplos da classe negativa
        neg_accuracy = accuracy_score(y_test[neg_mask], y_pred[neg_mask])
        neg_f1 = f1_score(y_test[neg_mask], y_pred[neg_mask], average='weighted')

        print(f"Acurácia para Negative: {neg_accuracy:.4f}")
        print(f"F1-Score para Negative: {neg_f1:.4f}")
    else:
        print("Nenhum exemplo da classe Negative encontrado no conjunto de teste.")

    # 11. Exemplos de erros (falsos negativos)
    false_negatives = X_test[ (y_test == 0) & (y_pred != 0) ]  # Corrigido para usar os índices originais
    if len(false_negatives) > 0:
        print("\nExemplos de falsos negativos:")
        #Recupera o texto original usando os indices originais.
        false_negatives_indices = np.where((y_test == 0) & (y_pred != 0))[0] # Obtem os indices dos falsos negativos.
        for i, index in enumerate(false_negatives_indices[:5]): # Usa os indices para acessar X_test
            print(f"{i+1}. {X.iloc[index]}") # Usa X.iloc para pegar os textos originais.
    else:
        print("Nenhum falso negativo encontrado.")
else:
    print("Nenhum modelo foi treinado (nenhuma combinação de parâmetros foi melhor).")

In [ ]:
import pandas as pd
import numpy as np
# Remova a importação de train_test_split se ela só for usada aqui
# from sklearn.model_selection import train_test_split
import time # Mantenha se usado no restante do script
from sklearn.model_selection import StratifiedKFold # Mantenha para a validação cruzada no treino
from sklearn.metrics import accuracy_score, f1_score, classification_report, recall_score # Mantenha para avaliação
# Importe suas classes e funções Keras/TensorFlow conforme necessário
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
# from tensorflow.keras.regularizers import l1, l2 # Importe se estiver usando regularização
# Certifique-se de que Model2VecEmbeddings e StaticModel estão acessíveis/importados
# from your_module import Model2VecEmbeddings, StaticModel
from typing import List # Necessário para a definição da classe mock/real

# --- Definição da sua classe Model2VecEmbeddings e StaticModel ---
# Copie/cole ou importe suas definições reais aqui se não estiverem disponíveis globalmente.
# Se já estiverem definidas/importadas, remova este bloco try/except com classes mock.
try:
    _ = Model2VecEmbeddings
    _ = StaticModel
except NameError:
    print("Model2VecEmbeddings or StaticModel not found. Using mock classes for demonstration.")
    # --- Mock classes for demonstration if yours aren't available ---
    class StaticModel:
        def __init__(self, model_name):
            print(f"Mock Loading model: {model_name}")
            self.embedding_size = 768 # Tamanho típico de embedding

        def encode(self, texts: List[str]):
            # print(f"Mock Encoding {len(texts)} texts...")
            return np.random.rand(len(texts), self.embedding_size).tolist()

        @classmethod
        def from_pretrained(cls, model_name):
            return cls(model_name)

    class Model2VecEmbeddings:
        """Wrapper para o Model2Vec como Embeddings - MOCK"""
        def __init__(self, model_name: str = "minishlab/potion-base-4M", similarity_threshold: float = 0.85):
             print(f"Mock Instantiating Model2VecEmbeddings: {model_name}")
             self.model = StaticModel.from_pretrained(model_name)
             self.similarity_threshold = similarity_threshold

        def embed_documents(self, texts: List[str]) -> List[List[float]]:
            return self.model.encode(texts).tolist()

        def embed_query(self, text: str) -> List[float]:
            return self.model.encode([text]).tolist()[0]
    # --- Fim das classes mock ---
# --- Fim da definição da classe (se necessário) ---


# 1. Carregar e preparar os dados de TREINO (do primeiro arquivo)
print("Carregando dados de treino do arquivo: /content/drive/MyDrive/Colab Notebooks/dataset_train_trim_synthetic_balanced.parquet")
df_train = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/dataset_train_trim_synthetic_balanced.parquet')

# Use 'target' ou 'sentiment' para os rótulos de treino
if 'target' not in df_train.columns:
    # Se 'target' não existe, use 'sentiment' como a coluna de rótulo
    train_label_column = 'sentiment'
    print("Coluna 'target' não encontrada no arquivo de treino. Usando 'sentiment' como coluna de rótulo.")
elif df_train['target'].dtype == 'object':
     # Se 'target' existe e é string, use 'target'
     train_label_column = 'target'
     print("Usando coluna 'target' (string) como rótulo no arquivo de treino.")
else:
    # Se 'target' existe e não é string (assumindo que já é int 0, 1, 2)
    train_label_column = 'target'
    print("Usando coluna 'target' (numérica) como rótulo no arquivo de treino.")


# Codificar labels (este mapeamento será usado para treino e teste)
sentiment_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}

# Aplicar o mapeamento para y_train
# Se a coluna de rótulo já for numérica (0, 1, 2), .map pode gerar NaNs ou não mudar nada,
# dependendo do conteúdo exato. É mais seguro verificar antes ou garantir que o mapeamento seja robusto.
# Assumindo que os rótulos no arquivo de treino são strings ('Negative', 'Neutral', 'Positive'):
y_train = df_train[train_label_column].map(sentiment_mapping)
X_train = df_train['comment_cleaned'] # Coluna de texto no arquivo de treino

# Verificação de rótulos de treino
print("Verificando rótulos de treino (y_train)...")
print(f"Tipo de dado de y_train: {y_train.dtype}")
print(f"Valores únicos em y_train: {y_train.unique()}")
if y_train.isnull().any():
    print("Aviso: NaNs encontrados em y_train após o mapeamento. Verifique os rótulos originais no dataset de treino.")


# 1. Carregar e preparar os dados de TESTE FINAL (do novo arquivo)
new_data_path = '/content/drive/MyDrive/Colab Notebooks/dataset_valid_with_sentiment_fix_negative.parquet'
print(f"\nCarregando dados de teste final do arquivo: {new_data_path}")
df_test_final = pd.read_parquet(new_data_path)

# Definir as colunas de texto e rótulo para o arquivo de teste.
# Ajuste se os nomes das colunas neste arquivo forem diferentes.
text_column_test = 'comment_cleaned'
sentiment_column_test = 'sentiment'

if text_column_test not in df_test_final.columns:
    print(f"Erro: Coluna de texto '{text_column_test}' não encontrada no dataset de teste final.")
    raise ValueError(f"Missing text column '{text_column_test}' in final test data.")
if sentiment_column_test not in df_test_final.columns:
     print(f"Erro: Coluna de sentimento '{sentiment_column_test}' não encontrada no dataset de teste final.")
     raise ValueError(f"Missing sentiment column '{sentiment_column_test}' in final test data.")

X_test_final = df_test_final[text_column_test]
y_test_final = df_test_final[sentiment_column_test]

# --- IMPORTANTE: GARANTIR QUE y_test_final ESTEJA NO FORMATO NUMÉRICO CORRETO (0, 1, 2) ---
# Aplique o mapeamento de rótulos para y_test_final usando o MESMO sentiment_mapping
# Se y_test_final já for numérico (int 0, 1, 2), a linha .map(inverse_sentiment_mapping)
# abaixo deve ser ajustada ou removida.
# Vamos verificar o tipo de dado e aplicar o mapeamento SE for string.

print("Verificando rótulos de teste final (y_test_final) do novo dataset...")
print(f"Tipo de dado de y_test_final: {y_test_final.dtype}")
print(f"Valores únicos em y_test_final: {y_test_final.unique()}")

# Codificar labels
sentiment_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
y_test_final = df_test_final[sentiment_column_test].map(sentiment_mapping)


# Neste ponto:
# X_train contém os textos do primeiro arquivo (para treino/VC)
# y_train contém os rótulos numéricos (0, 1, 2) do primeiro arquivo (para treino/VC)
# X_test_final contém os textos do segundo arquivo (para teste FINAL)
# y_test_final contém os rótulos numéricos (0, 1, 2) do segundo arquivo (para teste FINAL)
# sentiment_mapping mapeia 0, 1, 2 para 'Negative', 'Neutral', 'Positive' (para relatórios)
# df_test_final contém o dataframe completo do arquivo de teste final (útil para exemplos de erro)


# A variável X original (referenciando o texto completo do treino)
# pode ser mantida se outras partes do seu código a utilizarem,
# mas para mostrar exemplos de erro no conjunto de teste FINAL,
# você precisará usar df_test_final ou X_test_final.

# Para fins de compatibilidade com o seu código anterior que usava `X.iloc[index]`
# para mostrar exemplos de erro (que presumia que X continha todos os dados antes do split),
# vamos criar uma nova variável para o dataframe de treino completo se necessário,
# mas o ideal é adaptar a parte de erro para usar `df_test_final`
# já que você mostrará erros do *conjunto de teste final*.

# Removida a linha de train_test_split.
# As variáveis X_test e y_test do split aleatório NÃO SÃO MAIS USADAS.
# O restante do script precisará usar X_test_final e y_test_final
# na seção de avaliação final.

print("\nPreparação de dados concluída.")
print(f"Tamanho do conjunto de treino: {len(X_train)} exemplos.")
print(f"Tamanho do conjunto de teste final: {len(X_test_final)} exemplos.")


# --- O restante do seu código deve continuar a partir daqui ---
# O processo de vetorização/embedding de X_train para criar X_train_vec
# O processo de Grid Search/Validação Cruzada usando X_train_vec e y_train
# O treino do best_model final usando X_train_vec e y_train
# E então, a avaliação final usará X_test_final (para gerar X_test_final_vec) e y_test_final

Carregando dados de treino do arquivo: /content/drive/MyDrive/Colab Notebooks/dataset_train_trim_synthetic_balanced.parquet
Coluna 'target' não encontrada no arquivo de treino. Usando 'sentiment' como coluna de rótulo.
Verificando rótulos de treino (y_train)...
Tipo de dado de y_train: int64
Valores únicos em y_train: [0 1 2]

Carregando dados de teste final do arquivo: /content/drive/MyDrive/Colab Notebooks/dataset_valid_with_sentiment_fix_negative.parquet
Verificando rótulos de teste final (y_test_final) do novo dataset...
Tipo de dado de y_test_final: object
Valores únicos em y_test_final: ['Positive' 'Negative' 'Neutral']

Preparação de dados concluída.
Tamanho do conjunto de treino: 733 exemplos.
Tamanho do conjunto de teste final: 199 exemplos.


In [1]:
# 2. Vetorização do texto
print("Vetorizando textos...")
# vectorizer = TfidfVectorizer(
#     max_features=8000,
#     ngram_range=(1, 2),
#     stop_words='english'
# )
# X_train_vec = vectorizer.fit_transform(X_train)
# X_test_vec = vectorizer.transform(X_test)

# # Converter para arrays densos
# X_train_vec = X_train_vec.toarray()
# X_test_vec = X_test_vec.toarray()


embedding_model_instance = Model2VecEmbeddings(model_name="minishlab/potion-base-32M")

def get_mean_embedding(text):
    """
    Generates embedding for a single text using the pre-loaded Model2VecEmbeddings instance.
    Note: This function now uses the globally instantiated 'embedding_model_instance'.
    The name 'get_mean_embedding' might not be strictly accurate for all models,
    as it's now getting a single vector per text, not necessarily a mean of token vectors.
    """
    # Usa a instância do Model2VecEmbeddings e chama o método para um único texto
    # embed_query retorna uma lista de floats [float, float, ...]
    return embedding_model_instance.embed_query(text)

X_train_vec = np.array([get_mean_embedding(text) for text in X_train])
print(f"Length X_train_vec: {len(X_train_vec)}")
X_test_vec = np.array([get_mean_embedding(text) for text in X_test])

print(f"Length X_test_vec: {len(X_test_vec)}")

# 3. Função para criar modelo
def create_model(units1=64, units2=32, dropout_rate=0.3,
                learning_rate=0.001, optimizer='adam',
                activation='relu', regularizer=None):
    model = Sequential()
    # Exemplo de matriz de custos (penalizando mais falsos positivos para a classe 0 - Negativo)
    class_weights = {0: 5, 1: 1, 2: 1} # Peso 5 para a classe Negativo, 1 para as outras
    # Camada de entrada
    model.add(Dense(
        units=units1,
        input_dim=X_train_vec.shape[1],
        activation=activation,
        kernel_regularizer=regularizer
    ))
    model.add(Dropout(dropout_rate))

    # Camada oculta
    model.add(Dense(
        units=units2,
        activation=activation,
        kernel_regularizer=regularizer
    ))
    model.add(Dropout(dropout_rate))

    # Camada de saída
    model.add(Dense(3, activation='softmax'))

    # Compilar modelo
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    else:
        opt = RMSprop(learning_rate=learning_rate)

    model.compile(
        loss='sparse_categorical_crossentropy',  # Importante: para labels inteiros
        optimizer=opt,
        metrics=['accuracy']
    )

    return model

#Testando: units1=128, units2=32, dropout=0.2, lr=0.001, opt=adam, batch=32, epochs=20, activation=relu, regularizer=None

# 4. Configuração da Grid Search
print("\nConfigurando Grid Search...")
param_grid = {
    'units1': [128],
    'units2': [32],
    'dropout_rate': [0.2],
    'learning_rate': [0.001],
    'optimizer': ['adam'],
    'batch_size': [32],
    'epochs': [20],
    'activation': ['relu'],
    'regularizer': [None]
}

# 5. Grid Search Manual
print("\nIniciando Grid Search...")
start_time = time.time()

best_score = -1
best_params = {}
best_model = None

print(f"Length y_train {len(y_train)}")
# Para cada combinação de parâmetros
for units1 in param_grid['units1']:
    for units2 in param_grid['units2']:
        for dropout_rate in param_grid['dropout_rate']:
            for learning_rate in param_grid['learning_rate']:
                for optimizer_name in param_grid['optimizer']:
                    for batch_size in param_grid['batch_size']:
                        for epochs in param_grid['epochs']:
                            for activation in param_grid['activation']:
                                for regularizer_type in param_grid['regularizer']:
                                    print(f"\nTestando: units1={units1}, units2={units2}, dropout={dropout_rate}, lr={learning_rate}, opt={optimizer_name}, batch={batch_size}, epochs={epochs}, activation={activation}, regularizer={regularizer_type}")

                                    # Criar o modelo com os parâmetros atuais
                                    model = create_model(
                                        units1=units1,
                                        units2=units2,
                                        dropout_rate=dropout_rate,
                                        learning_rate=learning_rate,
                                        optimizer=optimizer_name,
                                        activation=activation,
                                        regularizer=regularizer_type
                                    )

                                    # Validação cruzada (cross-validation) - StratifiedKFold para manter a proporção das classes
                                    kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)  # Mantém a proporção das classes
                                    scores = []
                                    for train_index, val_index in kfold.split(X_train_vec, y_train):
                                        X_train_fold, X_val_fold = X_train_vec[train_index], X_train_vec[val_index]
                                        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usando .iloc para indexar corretamente
                                        # Treinar o modelo no conjunto de treinamento da dobra (fold)
                                        model.fit(X_train_fold, y_train_fold, batch_size=batch_size, epochs=epochs, verbose=0) # verbose=0 silencia a saída de treinamento

                                        # Fazer a predição no conjunto de validação da dobra
                                        y_pred_fold = np.argmax(model.predict(X_val_fold), axis=-1)  # Convertendo as predições de probabilidade para rótulos de classe
                                        # Calcular o F1-score ponderado para a dobra
                                        # f1 = f1_score(y_val_fold, y_pred_fold, average='weighted')
                                        # scores.append(f1)

                                        recall_negativo = recall_score(y_val_fold, y_pred_fold, average=None, labels=[0])[0]  # labels=[0] para o recall da classe 0 (Negativo)
                                        print(f"Recall (Negativo): {recall_negativo:.4f}")
                                        scores.append(recall_negativo)

                                    # Calcular a média dos scores de validação cruzada
                                    # mean_f1 = np.mean(scores)
                                    # print(f"F1-score médio (validação cruzada): {mean_f1:.4f}")

                                    mean_recall_negativo = np.mean(scores)
                                    print(f"Recall (Negativo) médio (validação cruzada): {mean_recall_negativo:.4f}")


                                    # Se o score for melhor, salvar os parâmetros e o modelo
                                    if mean_recall_negativo > best_score:
                                        best_score = mean_recall_negativo
                                        best_params = {
                                            'units1': units1,
                                            'units2': units2,
                                            'dropout_rate': dropout_rate,
                                            'learning_rate': learning_rate,
                                            'optimizer': optimizer_name,
                                            'batch_size': batch_size,
                                            'epochs': epochs,
                                            'activation': activation,
                                            'regularizer': regularizer_type
                                        }
                                        best_model = model # Salvando o modelo treinado com os melhores parâmetros
                                    print("-" * 30)  # Separador visual


print(f"\nTempo total de execução: {(time.time() - start_time) / 60:.2f} minutos")

# 7. Resultados
print("\nMelhores parâmetros encontrados:")
for param, value in best_params.items():
    print(f"{param}: {value}")

print(f"\nMelhor score de validação: {best_score:.4f}")

# 8. Avaliação no conjunto de teste (usando o melhor modelo treinado)
print("\nAvaliando no conjunto de teste...")

if best_model is not None:  # Certifique-se de que um modelo foi treinado
    y_pred = np.argmax(best_model.predict(X_test_vec), axis=-1) # Convertendo as predições de probabilidade para rótulos de classe
    test_accuracy = accuracy_score(y_test, y_pred)
    test_f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"\nAcurácia no teste: {test_accuracy:.4f}")
    print(f"F1-Score no teste: {test_f1:.4f}")

    # 9. Relatório detalhado
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred, target_names=sentiment_mapping.keys()))

    # 10. Análise por classe (especialmente para negativas)
    print("\nMétricas para classe Negative:")
    neg_mask = (y_test == 0)
    if np.any(neg_mask):  # Garante que haja exemplos da classe negativa
        neg_accuracy = accuracy_score(y_test[neg_mask], y_pred[neg_mask])
        neg_f1 = f1_score(y_test[neg_mask], y_pred[neg_mask], average='weighted')

        print(f"Acurácia para Negative: {neg_accuracy:.4f}")
        print(f"F1-Score para Negative: {neg_f1:.4f}")
    else:
        print("Nenhum exemplo da classe Negative encontrado no conjunto de teste.")

    # 11. Exemplos de erros (falsos negativos)
    false_negatives = X_test[ (y_test == 0) & (y_pred != 0) ]  # Corrigido para usar os índices originais
    if len(false_negatives) > 0:
        print("\nExemplos de falsos negativos:")
        #Recupera o texto original usando os indices originais.
        false_negatives_indices = np.where((y_test == 0) & (y_pred != 0))[0] # Obtem os indices dos falsos negativos.
        for i, index in enumerate(false_negatives_indices[:5]): # Usa os indices para acessar X_test
            print(f"{i+1}. {X.iloc[index]}") # Usa X.iloc para pegar os textos originais.
    else:
        print("Nenhum falso negativo encontrado.")
else:
    print("Nenhum modelo foi treinado (nenhuma combinação de parâmetros foi melhor).")

# Salvar o melhor modelo
if best_model is not None:
    # Criar um diretório para salvar o modelo (opcional)
    import os
    model_dir = "saved_models"
    os.makedirs(model_dir, exist_ok=True)

    # Definir o nome do arquivo do modelo
    model_filename = os.path.join(model_dir, "best_sentiment_model.h5")

    # Salvar o modelo
    best_model.save(model_filename)
    print(f"\nMelhor modelo salvo em: {model_filename}")

    # Se você também quiser salvar o vetorizador (embora no seu caso está usando embeddings)
    # Você pode querer salvar informações sobre o embedding_model_instance
    # Isso pode ser mais complexo dependendo da implementação do Model2VecEmbeddings
else:
    print("Nenhum modelo para salvar.")

Vetorizando textos...


NameError: name 'Model2VecEmbeddings' is not defined